## 2D-to-3D Lanes Pipeline
Created by Jett Penner<br>
December 2025 <br>


Run the custom 2d-to-3d rotation code and/or alternate rotations. Code currently set up for the ONCE-3DLanes dataset (with filetypes, etc); easy to modify to other datasets by changing data loaders.

Run after the `ONCE-3DLanes Data Loader`, and ensure that the config files match. Alter this config file to choose which type of thesis code to run.


In [ ]:
import yaml
import os
import numpy as np
from pathlib import Path
import open3d as o3d
import cv2
import json
import sys
from tqdm import tqdm
from collections import defaultdict
from scipy.interpolate import PchipInterpolator
from scipy.optimize import minimize_scalar
from logger import setup_logger, start_timer, stop_timer, switch_log_file

# Force single threading for accurate runtime analysis
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"   

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# Setup logger, global variable in other functions
with open("once" + "_config.yaml", "r") as f:
    once_config = yaml.safe_load(f)

logger = setup_logger("per_run_logger")
main_logger = setup_logger("summary_log")
switch_log_file(main_logger, once_config["output"]["base_path"])
main_logger.info("Starting Program\n")
log_specifics = False

### File Setup Functions

In [ ]:
def load_calibration(calibration_path, log=True):
    '''
    Load calibration parameters from a JSON file.

    Args:
        calibration_path (str): Path to calibration JSON file.
        log (bool): True if log information (set to false if not established, ie from other file).

    Returns:
        tuple: (
            T_camera_lidar (4,4) np array, 
            T_applanix_lidar (4,4) np array,
            fx, fy, cx, cy (float),
            img_width, img_height (int),
            distortion (5,) np array
        )
    '''
    try:
        with open(calibration_path, 'r') as f:
            calibration_data = json.load(f)
        if log and log_specifics:
            logger.info(f"Loaded calibration file: {calibration_path}")

        # Parse transformation matrices
        T_camera_lidar = np.array(calibration_data["T_camera_lidar"], dtype=float)
        T_applanix_lidar = np.array(calibration_data["T_applanix_lidar"], dtype=float)

        # Parse intrinsics
        intrinsics = calibration_data["camera_intrinsics"]
        fx = float(intrinsics["fx"])
        fy = float(intrinsics["fy"])
        cx = float(intrinsics["cx"])
        cy = float(intrinsics["cy"])
        distortion = np.array(intrinsics["distortion"], dtype=float)

        # Parse image parameters
        img_params = calibration_data["image_params"]
        img_width = int(img_params["width"])
        img_height = int(img_params["height"])

        return (
            T_camera_lidar,
            T_applanix_lidar,
            fx, fy, cx, cy,
            img_width, img_height,
            distortion
        )

    except FileNotFoundError:
        if log and log_specifics:
            logger.error(f"Calibration file not found: {calibration_path}")
        else:
            print(f"Calibration file not found: {calibration_path}")
        sys.exit(1)
    except KeyError as e:
        if log and log_specifics:
            logger.error(f"Missing key in calibration file ({e}) — check structure of {calibration_path}")
        else:
            print(f"Missing key in calibration file ({e}) — check structure of {calibration_path}")
        sys.exit(1)
    except Exception as e:
        if log and log_specifics:
            logger.error(f"Error reading calibration file {calibration_path}: {e}")
        else:
            print(f"Error reading calibration file {calibration_path}: {e}")
        sys.exit(1)



def load_2d_lanes(lane_detection_path, config, img_width, img_height, flip_y=True, log=True):
    '''
    Load and convert 2D lane detections from a .txt file into pixel coordinates.

    Args:
        lane_detection_path (str): Path to the lane detection .txt file.
        config (dict): Configuration file.
        img_width, img_height (int): Camera mage width and height.
        flip_y (bool, optional): If true, flips y-coordinates to image origin (top-left).
        log (bool): True if log information (set to false if not established, ie from other file).

    Returns:
        list[np array]: Each element is an (N,2) np array of integer pixel coordinates [x, y] for one lane.
    '''
    detection_width = config["lane_detection_config"]["detection_width"]
    detection_height = config["lane_detection_config"]["detection_height"]
    if detection_width is None or detection_height is None:
        if detection_width is None and detection_height is None:
            detection_width = img_width
            detection_height = img_height
    scale_x = img_width / detection_width
    scale_y = img_height / detection_height
    lanes = []
    try:
        with open(lane_detection_path, "r") as f:
            for l in f:
                l = l.strip()
                if " confidence:" in l:
                    parts = l.rsplit(" confidence:", 1)
                    lane_coords = [(float(x), float(y)) for x, y in (pair.split(',') for pair in parts[0].split())]
                    lane_x, lane_y = zip(*lane_coords)
                    lane_x = [float(x * scale_x) for x in lane_x]
                    lane_y = [float(y * scale_y) for y in lane_y]
                    lane_x = np.asarray(lane_x, dtype=int)
                    lane_y = np.asarray(lane_y, dtype=int)
                    if np.any(lane_x < 0) or np.any(lane_y < 0):
                        if log and log_specifics:
                            logger.error("Lane anchors have a negative pixel coordinate, error from ONCE")
                        else:
                            print("Lane anchors have a negative pixel coordinate, error from ONCE")
                        sys.exit(1)
                    if flip_y:
                        lane_y = img_height - lane_y
                    lanes.append(np.hstack([lane_x.reshape(-1,1), lane_y.reshape(-1,1)]))
    except FileNotFoundError:
        if log and log_specifics:
            logger.warning(f"lane detection file not found: {lane_detection_path}")

    return lanes



def get_lidar_points(lidar_file_path):
    '''
    Load a LiDAR binary file and extract point data.

    Args:
        lidar_file_path (str): Path to LiDAR .bin file.

    Returns:
        tuple: (
            points_xyz (N,3) np array,
            intensity (N,) np array,
            ring (N,) np array,
            time (N,) np array
        )
    '''
    points = np.fromfile(lidar_file_path, dtype=np.float32).reshape(-1, 6)
    x, y, z, intensity, ring, time = points.T
    points_xyz = np.vstack((x, y, z)).T

    return points_xyz, intensity, ring, time



def prefilter_ground_plane(points, sensor_height=1.5, height_range=None, vertical_axis=0):
    """
    Prefilter a point cloud to keep only points near the expected ground plane height.

    Args:
        points (N,3) np array: Input 3D point cloud.
        sensor_height (float): Expected height of the sensor above the ground plane (approximate).
        height_range (float): Acceptable deviation (±) from the expected ground level.
        vertical_axis (int): Axis index representing "up" direction (0=x, 1=y, 2=z).

    Returns:
        (M,3) np array: Filtered point cloud containing only points near ground level.
    """
    if points is None or len(points) == 0:
        return np.empty((0, 3), dtype=np.float32)
    
    if height_range is None:
        height_range = sensor_height/2

    min_height = -sensor_height - height_range
    max_height = -sensor_height + height_range

    # Select points whose coordinate along the vertical axis is within range
    mask = (points[:, vertical_axis] >= min_height) & (points[:, vertical_axis] <= max_height)
    filtered_points = points[mask]

    return filtered_points.astype(np.float32)



def estimate_ground_plane(
    points,
    config,
    vertical_axis=0,
    random_state=None
):
    '''
    Estimate the ground plane from 3D points using a band filter + RANSAC + least-squares refit.

    Args:
        points (N,3) np array: Input 3D point cloud.
        config (dict): Configuration file.
        vertical_axis (int, optional): Axis index representing up direction. Default is 0 (x-axis).
        random_state (int, optional): Seed for random generator.

    Returns:
        tuple: (
            plane_normal (3,) np array: Unit normal vector.
            plane_offset (float): Plane offset d in n·x + d = 0.
            inlier_mask (N,3) np array[bool]: Mask of inlier points.
        )
    '''

    sensor_height = config["ground_plane_ransac"]["camera_height"]
    band_half_width = config["ground_plane_ransac"]["band_half_width"]
    ransac_iters = config["ground_plane_ransac"]["ransac_iters"]
    inlier_threshold = config["ground_plane_ransac"]["inlier_threshold"]
    min_inliers_for_accept = config["ground_plane_ransac"]["min_inliers_for_accept"]
    inlier_fallback_range_mult = config["ground_plane_ransac"]["inlier_fallback_range_mult"]
    plane_norm_degenerate_threshold = float(config["ground_plane_ransac"]["plane_norm_degenerate_threshold"])

    if random_state is None:
        rng = np.random.default_rng()
    else:
        rng = np.random.default_rng(random_state)

    pts = np.asarray(points)

    # expected ground coordinate value is  -sensor_height.
    ground_coord = -sensor_height

    use_fallback_plane=False

    if pts.size == 0:
        use_fallback_plane=True
        logger.error("No points in lidar ransac around ground coordinate after prefunction filter")
    else:

        # Pre-filter: keep points within a vertical band around expected ground
        vert_vals = pts[:, vertical_axis]
        band_mask = np.logical_and(vert_vals >= (ground_coord - band_half_width),
                                vert_vals <= (ground_coord + band_half_width))

        candidate_idx = np.nonzero(band_mask)[0]
        if candidate_idx.size < min_inliers_for_accept:
            # too few candidates - relax band slightly (quick fallback)
            band_mask = np.logical_and(vert_vals >= (ground_coord - inlier_fallback_range_mult * band_half_width),
                                    vert_vals <= (ground_coord + inlier_fallback_range_mult * band_half_width))
            candidate_idx = np.nonzero(band_mask)[0]

        
        if candidate_idx.size < 3:
            # not enough points to fit a plane
            use_fallback_plane=True
            logger.error(f"Not enough candidate points for lidar ransac around ground coordinate after infunction filter: need 3, got {candidate_idx.size}")

    if not use_fallback_plane:
        cand_pts = pts[candidate_idx]

        best_inliers = None
        best_count = 0

        # RANSAC loop: sample 3 points, form plane, count inliers
        M = cand_pts.shape[0]
        for _ in range(ransac_iters):
            ids = rng.choice(M, size=3, replace=False)
            p0, p1, p2 = cand_pts[ids]

            v1 = p1 - p0
            v2 = p2 - p0
            n = np.cross(v1, v2)
            norm_n = np.linalg.norm(n)
            if norm_n < plane_norm_degenerate_threshold:
                continue  # degenerate sample, skip

            n = n / norm_n
            d = -np.dot(n, p0)

            distances = np.abs(cand_pts.dot(n) + d)

            # count inliers
            mask_in = distances <= inlier_threshold
            count = int(mask_in.sum())

            if count > best_count:
                best_count = count
                best_inliers = candidate_idx[mask_in]  # indices into original pts

    # require minimum inliers or not confident
    if use_fallback_plane or best_count < min_inliers_for_accept:
        if best_count < min_inliers_for_accept:
            logger.warning(f"Not enough points on best ground plane to surpass the min_inliers threshold: needed {min_inliers_for_accept}, got {best_count}")
        logger.warning("Using fallback constant-height plane")
        # return fallback: default flat plane normal pointing up
        fallback_n = np.zeros(3)
        fallback_n[vertical_axis] = 1.0
        fallback_d = -ground_coord
        if pts.size == 0:
            inlier_mask = np.zeros(0, dtype=bool)
        else:
            inlier_mask = band_mask  # treat band as inliers
        return fallback_n, fallback_d, inlier_mask


    # Refit plane with all best inliers using SVD
    inlier_pts = pts[best_inliers]
    centroid = inlier_pts.mean(axis=0)
    cov = inlier_pts - centroid
    _, _, vh = np.linalg.svd(cov, full_matrices=False)
    normal = vh.T[:, -1]
    # enforce normal to point up (structure priors)
    if normal[vertical_axis] < 0:
        normal = -normal
    d = -np.dot(normal, centroid)

    # compute final inlier mask (on the whole cloud) using threshold
    distances_all = np.abs(pts.dot(normal) + d)
    inlier_mask = distances_all <= inlier_threshold

    if d > 0:
        normal = -normal
        d = -d

    return normal, d, inlier_mask



def filter_and_clip_points(
    points_camera,
    fx, fy, cx, cy,
    img_width, img_height,
    config,
    distortion=None
):
    """
    Projects 3D camera-frame points into image plane and filters only those that fall 
    inside the camera's (distorted) field of view.

    Args:
        points_camera (N,3) np array: 3D points in camera frame.
        fx, fy, cx, cy (float): Intrinsic parameters.
        img_width, img_height (int): Image dimensions.
        config (dict): Configuration file.
        distortion (5,) optional np array: [k1, k2, p1, p2, k3] distortion coefficients.

    Returns:
        u (M,) np array: x-pixel coordinates visible on the distorted FOV.
        v (M,) np array: y-pixel coordinates visible on the distorted FOV.
        d (M,) np array: depths along the viewing ray.
        points_3d_visible (M,3) np array: Corresponding 3D points visible in the distorted FOV.
    """
    depth_cutoff_min = config["depth_map"]["depth_cutoff_min"]
    depth_cutoff_max = config["depth_map"]["depth_cutoff_max"]
    depth_max_point_scaling_distance = config["depth_map"]["depth_max_point_scaling_distance"]
    lidar_point_size_min = config["depth_map"]["lidar_point_size_min"]
    lidar_point_size_max = config["depth_map"]["lidar_point_size_max"]

    X, Y, Z = points_camera[:, 0], points_camera[:, 1], points_camera[:, 2]
    Z = np.maximum(Z, 1e-6)  # avoid divide-by-zero

    K = np.array([[fx, 0, cx],
                  [0, fy, cy],
                  [0,  0, 1]], dtype=np.float64)

    # Project with distortion
    rvec = np.zeros((3, 1), dtype=np.float64)
    tvec = np.zeros((3, 1), dtype=np.float64)
    points_2d, _ = cv2.projectPoints(points_camera, rvec, tvec, K, distortion)
    points_2d = points_2d.reshape(-1, 2)
    u, v = points_2d[:, 0], points_2d[:, 1]

    # Visibility mask
    mask = (
        (Z > 0) &
        (u >= 0) & (u < img_width) &
        (v >= 0) & (v < img_height) &
        (Z >= depth_cutoff_min) & (Z <= depth_cutoff_max)
    )
    points_3d_visible = points_camera[mask]
    points_2d_visible = points_2d[mask]
    c = Z[mask]
    u = points_2d_visible[:, 0]
    v = points_2d_visible[:, 1]
    v_flipped = img_height - v

    depth_clipped = np.clip(c, depth_cutoff_min, depth_max_point_scaling_distance)  # Clip to fixed range
    depth_normalized = (depth_clipped - depth_cutoff_min) / (depth_max_point_scaling_distance - depth_cutoff_min)
    sizes = lidar_point_size_max - (depth_normalized * (lidar_point_size_max - lidar_point_size_min))  # Linear interpolation

    return u, v_flipped, c, sizes, points_3d_visible



def create_spatial_bins(u, v, r, depth, pts, cell_size):
    '''
    Create spatial bins (grid cells) for 2D pixel / depth data for fast searching.

    Args:
        u (N,) np array: x pixel coordinates.
        v (N,) np array: y pixel coordinates.
        r (N,) np array: Point render radius (px)
        depth (N,) np array: Point depth.
        pts (N,3) np array: Associated 3D points.
        cell_size (float): Size of each grid cell.

    Returns:
        dict: {
            bins (dict): the dictionary of bins.
            
            All inputs.
        }
    '''
    # Create integer grid keys for each point
    cell_x = np.floor(u / cell_size).astype(int)
    cell_y = np.floor(v / cell_size).astype(int)

    bins = defaultdict(list)

    for i in range(len(u)):
        bins[(cell_x[i], cell_y[i])].append(i)

    return {
        "bins": bins,
        "u": u,
        "v": v,
        "r": r,
        "depth": depth,
        "pts": pts,
        "cell_size": cell_size
    }



def fit_line_on_plane_precalc(plane_normal, plane_offset):
    '''
    Precompute plane basis vectors and origin for efficient repeated line fitting.

    Args:
        plane_normal (3,) np array: Unit normal vector.
        plane_offset (float): Plane offset d in n·x + d = 0.

    Returns:
        dict: {
            plane_x (3,) np array: Orthonormal basis (unit) vector lying on the x axis.
            plane_y (3,) np array: Orthonormal basis (unit) vector lying on the y axis.
            origin (3,) np array: 3D point lying on the plane (satisfies n·origin + d = 0).
        }
    '''
    n = plane_normal / np.linalg.norm(plane_normal)
    # Create local 2D basis vectors
    arbitrary = np.array([1, 0, 0]) if abs(n[0]) < 0.9 else np.array([0, 1, 0])
    plane_x = np.cross(n, arbitrary)
    plane_x /= np.linalg.norm(plane_x)
    plane_y = np.cross(n, plane_x)


    origin = -plane_offset * n  # any point that satisfies n·x + d = 0

    return {
        "plane_x": plane_x, 
        "plane_y": plane_y, 
        "origin": origin
    }


### Per Line Functions

In [ ]:
def pure_intrinsic_projection(
    u, v,
    fx, fy, cx, cy, 
    config,
    distortion = None,
    height_cam = None,
    plane_normal = None,
    plane_offset = None
):
    """
    Project pixels (u, v) to 3D positions in camera frame, either onto a flat ground or an arbitrary plane.

    Args:
        u (N,) np array: x pixel coordinates.
        v (N,) np array: y pixel coordinates.
        fx, fy, cx, cy (float): Intrinsic parameters.
        distortion (5,) optional np array: [k1, k2, p1, p2, k3] distortion coefficients.
        
        Need either:
            1) height_cam (float, optional): if given, project rays onto flat ground at Y = -height_cam.
            2) plane_normal (3,) optional np array: if given, project rays onto unit normal of plane.
               plane_offset (float, optional): Plane offset d in n·x + d = 0.

    Returns:
        points_3d (N,3) np array: 3D points in camera frame.
    """
    if height_cam is None and (plane_normal is None or plane_offset is None):
        if log_specifics:
            logger.error("Must provide either height_cam or plane_normal + plane_offset")
        else:
            print("Must provide either height_cam or plane_normal + plane_offset")
        sys.exit(1)
    
    parallel_plane_thresh = float(config["intrinsic_plane_projection"]["parallel_plane_thresh"])
    u = np.asarray(u, dtype=np.float64)
    v = np.asarray(v, dtype=np.float64)

    # Pinhole
    x = (u - cx) / fx
    y = (v - cy) / fy

    # Distortion (manual, proof of working math for thesis)
    if distortion is not None:
        k1, k2, p1, p2, k3 = distortion[:5]
        r2 = x**2 + y**2
        radial = 1 + k1*r2 + k2*r2**2 + k3*r2**3
        x_distorted = x * radial + 2*p1*x*y + p2*(r2 + 2*x**2)
        y_distorted = y * radial + p1*(r2 + 2*y**2) + 2*p2*x*y
        x, y = x_distorted, y_distorted

    rays = np.stack([x, y, np.ones_like(x)], axis=-1)
    points_3d = np.zeros_like(rays)

    if height_cam is not None:
        # Flat ground at Y = -height_cam
        denom = rays[:, 1]  # cam frame, Y is vertical axis
        valid = np.abs(denom) > parallel_plane_thresh
        num_invalid = np.count_nonzero(~valid)
        if num_invalid > 0:
            logger.error(f"Intrinsic proj to plane:{num_invalid} invalid (parallel) rays")
        s = np.zeros_like(denom)
        s[valid] = -height_cam / denom[valid]
        points_3d[valid] = rays[valid] * s[valid, None]
        points_3d[~valid] = np.nan  # ray parallel to ground
    else:
        # Arbitrary plane: plane_normal · X + plane_offset = 0
        plane_normal = np.asarray(plane_normal, dtype=np.float64)
        denom = rays @ plane_normal
        valid = np.abs(denom) > parallel_plane_thresh
        num_invalid = np.count_nonzero(~valid)
        if num_invalid > 0:
            logger.error(f"Intrinsic proj to plane:{num_invalid} invalid (parallel) rays")
        s = np.zeros_like(denom)
        s[valid] = -plane_offset / denom[valid]
        points_3d[valid] = rays[valid] * s[valid, None]
        points_3d[~valid] = np.nan  # ray parallel to plane

    return points_3d



def fit_line_on_plane(points, precalc_values, curvature_threshold=0.05):
    """
    Fit a 3D line (or quadratic curve if curvature is high) through points known to lie on a plane.

    Args:
        points (N,3) np array: 3D points (already on plane).
        precalc_values (dict): dict containing plane_x, plane_y, origin (see fit_line_on_plane_precalc()).
        curvature_threshold (float, optional): RMS threshold to switch to quadratic fitting. Default is 0.05.

    Returns:
        tuple: (
            model_type (str): the used fitting model, either "linear" or "quadratic",
            model_params (dict): {
                coeffs (tuple) of floats: 
                    Linear: (a,b) satisfying a·x + b = 0,
                    Quadratic: (a,b,c) satisfying a·x^2 + b·x + c = 0.
                z_intercept (3,) np array: 3D point on plane for the y-intercept.
                
                line_dir:
                    Linear: (3,) np array: Unit direction vector along the fitted line.
                    Quadratic: None
                quad_firstorder_dir:
                    Linear: None
                    Quadratic: (3,) np array: Unit direction vector for first-order change along the fitted line.
                quad_secondorder_dir:
                    Linear: None
                    Quadratic: (3,) np array: Unit direction vector for second-order change along the fitted line.
            }
        )
    """
    plane_x = precalc_values["plane_x"]
    plane_y = precalc_values["plane_y"]
    origin = precalc_values["origin"]

    # Convert points to 2D local coordinates on plane
    pts = np.asarray(points)
    local_pts = pts - origin
    x = local_pts @ plane_x
    y = local_pts @ plane_y

    # Fit linear model y = a*x + b
    A = np.vstack([x, np.ones_like(x)]).T
    a, b = np.linalg.lstsq(A, y, rcond=None)[0]
    y_pred = a * x + b
    linear_rms = np.sqrt(np.mean((y - y_pred) ** 2))

    # Quadratic fallback if large rms error
    if linear_rms > curvature_threshold:
        a2, b2, c2 = np.polyfit(x, y, 2)

        # Fit quadratic model y = a*(x^2) + b*x + c
        quad_origin = origin + c2 * plane_y
        quad_firstorder_dir = plane_x + b2 * plane_y
        quad_secondorder_dir = a2 * plane_y

        if np.dot(quad_firstorder_dir, np.array([0, 0, 1])) < 0:
            quad_firstorder_dir *= -1

        if np.dot(quad_secondorder_dir, np.array([0, 0, 1])) < 0:
            quad_secondorder_dir *= -1

        return "quadratic", {
            "coeffs": (a2, b2, c2),
            "z_intercept": quad_origin,
            "line_dir": None,
            "quad_firstorder_dir": quad_firstorder_dir,
            "quad_secondorder_dir": quad_secondorder_dir
        }

    # Return linear
    line_dir = plane_x + a * plane_y
    if np.dot(line_dir, np.array([0, 0, 1])) < 0:
        line_dir *= -1
    line_origin = origin + b * plane_y

    return "linear", {
        "coeffs": (a, b),
        "z_intercept": line_origin,
        "line_dir": line_dir / np.linalg.norm(line_dir),
        "quad_firstorder_dir": None,
        "quad_secondorder_dir": None
    }



def query_nearest_depth_from_bins(bins_info, x, y, bin_search_range=5):
    """
    Query the nearest (u, v, depth, point) for the given pixel (x, y)
    by expanding search radius across bins until a match is found.

    Args:
        bins_info (dict): dict containing bins, cell_size, u, v, depth, pts (see create_spatial_bins()).
        x (float): x pixel coordinate to query.
        y (float): y pixel coordinate to query.
        bin_search_range: how many neighboring bins (in Manhattan distance) to search outward from the current bin. 
            If None, expands outward until a match is found.

    Returns:
        tuple (
            u_match (float): x pixel coordinate of match, 
            v_match (float): y pixel coordinate of match,
            depth_match (float): Depth value of match,
            pt_match (3,) np array: 3D point coordinate of match
        )
        or (None, None, None, None) if no match found.
    """
    bins = bins_info["bins"]
    cell_size = bins_info["cell_size"]
    u = bins_info["u"]
    v = bins_info["v"]
    depth = bins_info["depth"]
    pts = bins_info["pts"]

    if not bins:
        if log_specifics:
            logger.error("Bins is undefined on access")
        else: 
            print("Bins is undefined on access")
        return None, None, None, None

    # Identify base bin cell
    key_x = int(np.floor(x / cell_size))
    key_y = int(np.floor(y / cell_size))

    best_idx = None
    best_dist2 = np.inf

    # Determine if infinite search (only for alt solutions)
    infinite_search = bin_search_range is None
    if infinite_search:
        bin_search_range = 0  # start at current bin

    visited_bins = set()
    search_r = 0 if infinite_search else 1
    while True:
        matched_any = False
        for dx in range(-search_r, search_r + 1):
            for dy in range(-search_r, search_r + 1):
                # Skip inner bins for r > 0
                if search_r > 0 and abs(dx) < search_r and abs(dy) < search_r:
                    continue

                cell_key = (key_x + dx, key_y + dy)
                if cell_key in visited_bins or cell_key not in bins:
                    continue
                visited_bins.add(cell_key)

                indices = bins[cell_key]
                if not indices:
                    continue

                matched_any = True
                u_i = u[indices]
                v_i = v[indices]
                dists2 = (u_i - x) ** 2 + (v_i - y) ** 2

                min_local_idx = np.argmin(dists2)
                if dists2[min_local_idx] < best_dist2:
                    best_dist2 = dists2[min_local_idx]
                    best_idx = indices[min_local_idx]

        # Stop when any match found
        if matched_any and best_idx is not None:
            break

        if infinite_search:
            # Stop when we've checked all bins
            if len(visited_bins) == len(bins):
                break
        else:
            # Stop when we've checked all bins in max range
            if search_r >= bin_search_range:
                break

        # Expand search radius
        search_r += 1

    if best_idx is None:
        if infinite_search and len(visited_bins) == len(bins):
            if log_specifics:
                logger.error("All bins checked, no matched points")
            else:
                print ("All bins checked, no matched points")
        return None, None, None, None

    return (
        float(u[best_idx]),
        float(v[best_idx]),
        float(depth[best_idx]),
        pts[best_idx],
    )



def project_pixel_to_depth(u, v, d, fx, fy, cx, cy, distortion=None):
    """
    Backproject pixel coordinates (u, v) with depth d into 3D camera coordinates.
    Undistorts pixels when distortion coefficients are provided.

    Args:
        u (float or (N,) np array): x pixel coordinate(s).
        v (float or (N,) np array): y pixel coordinate(s).
        d (float or (N,) np array): Depth value(s).
        fx, fy, cx, cy (float): Intrinsic parameters.
        distortion (5,) optional np array: [k1, k2, p1, p2, k3] distortion coefficients.

    Returns:
        points_3d (float or (N,3) np array): 3D point(s) in camera frame.
    """
    # Convert inputs to 1D arrays
    u = np.atleast_1d(np.asarray(u, dtype=np.float64))
    v = np.atleast_1d(np.asarray(v, dtype=np.float64))
    d = np.atleast_1d(np.asarray(d, dtype=np.float64))

    if not (u.shape == v.shape == d.shape):
        raise ValueError("u, v, d must have the same shape")

    N = u.size
    if N == 0:
        return np.zeros((0, 3), dtype=np.float64)

    # Build points array in shape (N,1,2)
    pts = np.stack([u, v], axis=-1).astype(np.float64)
    pts_cv = pts.reshape(-1, 1, 2)

    if distortion is not None:
        K = np.array([[fx, 0, cx],
                      [0, fy, cy],
                      [0,  0, 1]], dtype=np.float64)
        undist = cv2.undistortPoints(pts_cv, K, distortion, P=K)
        undist = undist.reshape(-1, 2)
        x = (undist[:, 0] - cx) / fx
        y = (undist[:, 1] - cy) / fy
    else:
        x = (u - cx) / fx
        y = (v - cy) / fy

    # Backproject to 3D (camera frame)
    X = x * d
    Y = y * d
    Z = d

    pts3d = np.stack((X, Y, Z), axis=-1)

    return pts3d



def query_depth_from_bins(bins_info, x, y, get_all_depths=False, prioritize_closest=True):
    '''
    Query nearby binned points for a given pixel location. Only looks in the pixel's bin.

    Args:
        bins_info (dict): dict containing bins, cell_size, u, v, depth, pts (see create_spatial_bins()).
        x (float): x pixel coordinate to query.
        y (float): y pixel coordinate to query.
        get_all_depths (bool, optional): If True, return all matched depths and points. 
            Default is False, going to prioritize_closest logic.
        prioritize_closest (bool, optional): If True, prefer the match nearest in 2D distance; otherwise, by smallest depth.

    Returns:
        tuple: (
            x pixel coordinate(s) (float or (N,) np array),
            y pixel coordinate(s) (float or (N,) np array),
            Depth value(s) (float or (N,) np array),
            3D matched point(s) ((3,) or (N,3) np array)
        )
        or (None, None, None, None) if no match found.
    '''
    bins = bins_info["bins"]
    cell_size = bins_info["cell_size"]
    u = bins_info["u"]
    v = bins_info["v"]
    r = bins_info["r"]
    depth = bins_info["depth"]
    pts = bins_info["pts"]

    # Identify which grid cell (x,y) lies in
    key_x = int(np.floor(x / cell_size))
    key_y = int(np.floor(y / cell_size))

    matched_depths = []
    matched_points = []
    us = []
    vs = []

    # Search the 3x3 neighborhood
    for dx in (-1, 0, 1):
        for dy in (-1, 0, 1):
            cell_key = (key_x + dx, key_y + dy)
            if cell_key not in bins:
                continue

            indices = bins[cell_key]
            if not indices:
                continue

            u_i = u[indices]
            v_i = v[indices]
            r_i = r[indices]

            # Box test
            dxs = np.abs(u_i - x)
            dys = np.abs(v_i - y)
            mask_box = (dxs <= r_i) & (dys <= r_i)
            if not np.any(mask_box):
                continue

            # Circle membership test
            dxs = dxs[mask_box]
            dys = dys[mask_box]
            r_i = r_i[mask_box]

            mask_circle = (dxs**2 + dys**2) <= r_i**2
            if not np.any(mask_circle):
                continue

            matched_depths.extend(depth[indices][mask_box][mask_circle])
            matched_points.extend(pts[indices][mask_box][mask_circle])
            us.extend(u_i[mask_box][mask_circle])
            vs.extend(v_i[mask_box][mask_circle])

    if not matched_depths:
            return None, None, None, None

    if get_all_depths:
        return us, vs, matched_depths, matched_points

    matched_depths = np.array(matched_depths)
    us = np.array(us)
    vs = np.array(vs)

    if prioritize_closest:
        # Choose the match with the smallest 2D distance to (x, y)
        dists2 = (np.array(us) - x)**2 + (np.array(vs) - y)**2
        min_idx = np.argmin(dists2)
    else:
        # Choose the match with smallest depth (closest to camera)
        min_idx = np.argmin(matched_depths)

    return us[min_idx], vs[min_idx], float(matched_depths[min_idx]), matched_points[min_idx]



def verify_with_intrinsic_projection(
    u, v,
    fx, fy, cx, cy, 
    config,
    plane_normal,
    plane_offset,
    verify_depth,
    distortion=None,
):
    """
    Determine if a depth is valid, thus non-parallel and within a maximum allowable range.

    Args:
        u (float): x pixel coordinate.
        v (float): y pixel coordinate.
        fx, fy, cx, cy (float): Intrinsic parameters.
        config (dict): Configuration file.
        plane_normal (3,) np array: Unit normal vector.
        plane_offset (float): Plane offset d in n·x + d = 0.
        verify_depth (float): The depth value to verify.
        distortion (5,) optional np array: [k1, k2, p1, p2, k3] distortion coefficients.

    Returns:
        (float): the verified depth, or the intrinsic depth if it failed the threshold check.
    """
    parallel_plane_thresh = float(config["intrinsic_plane_projection"]["parallel_plane_thresh"])
    max_deviation = config["depth_map"]["max_deviation_from_intr"]

    x = (u - cx) / fx
    y = (v - cy) / fy

    if distortion is not None:
        k1, k2, p1, p2, k3 = distortion[:5]
        r2 = x**2 + y**2
        radial = 1 + k1*r2 + k2*r2**2 + k3*r2**3
        x = x * radial + 2*p1*x*y + p2*(r2 + 2*x**2)
        y = y * radial + p1*(r2 + 2*y**2) + 2*p2*x*y

    ray = np.array([x, y, 1.0], dtype=np.float64)
    denom = ray @ np.asarray(plane_normal, dtype=np.float64)
    
    if np.abs(denom) <= parallel_plane_thresh:
        return verify_depth

    s = abs(-plane_offset / denom)

    if verify_depth > s * (1 + max_deviation):
        return s
    
    return verify_depth



def cumulative_lengths_2d(pts2d):
    '''
    Compute cumulative arc lengths along a 2D polyline.

    Args:
        pts2d (N,2) np array: 2D points.

    Returns:
        Cumulative lengths (N,) np array with first element = 0.0.
    '''
    diffs = np.diff(pts2d, axis=0)
    seglen = np.linalg.norm(diffs, axis=1)
    return np.concatenate(([0.0], np.cumsum(seglen)))



def project_point_to_2d_polyline(pt, poly):
    '''
    Project a 2D point onto a 2D polyline.

    Args:
        pt (2,) np array: 2D point.
        poly (N,2) np array: Polyline points.

    Returns:
        tuple: (
            Cumulative distance along polyline at projection (float),
            Projected point on polyline (2,) np array,
            Index of segment used (int),
            Fractional position along the segment (float),
            Euclidean distance from point to projection (float)
        )
    '''
    best_dist = np.inf
    best = None
    cum = cumulative_lengths_2d(poly)
    seg_starts = poly[:-1]
    seg_ends = poly[1:]
    seg_vecs = seg_ends - seg_starts
    seg_lens2 = np.sum(seg_vecs**2, axis=1)
    for i, (p0, v, L2) in enumerate(zip(seg_starts, seg_vecs, seg_lens2)):
        if L2 == 0:
            frac = 0.0
            proj = p0
        else:
            t = np.dot(pt - p0, v) / L2
            frac = np.clip(t, 0.0, 1.0)
            proj = p0 + frac * v
        d = np.linalg.norm(pt - proj)
        s_here = cum[i] + frac * np.sqrt(L2)
        if d < best_dist:
            best_dist = d
            best = (s_here, proj, i, frac, d)
    return best



def lane_point_linear_model(linear_intercept, linear_dir, t):
    '''
    Compute 3D lane points along a linear parametric model: P(t) = linear_intercept + t * linear_dir.

    Args:
        linear_intercept (3,) np array: 3D origin point of the lane in the reference frame.
        linear_dir (3,) np array: Unit direction vector of the lane.
        t (float (N,) np array): Scalar(s) representing distance parameter along the lane.

    Returns:
        3D lane points corresponding to t ((3,) or (N,3) np array)
    '''
    t = np.asarray(t)    # t can be scalar or array
    return linear_intercept + np.outer(t, linear_dir)



def lane_point_quadratic_model(quadratic_intercept, quadratic_firstorder_dir, quadratic_secondorder_dir, t):
    '''
    Compute 3D lane points along a quadratic parametric model: P(t) = quadratic_intercept 
        + t * quadratic_firstorder_dir + t² * quadratic_secondorder_dir.

    Args:
        quadratic_intercept (3,) np array: 3D base point of the lane curve.
        quadratic_firstorder_dir (3,) np array: First-order unit direction vector.
        quadratic_secondorder_dir (3,) np array: Second-order unit curvature vector/
        t (float (N,) np array): Scalar(s) representing distance parameter along the lane.

    Returns:
        3D lane points corresponding to t ((3,) or (N,3) np array)
    '''
    t = np.asarray(t)
    return quadratic_intercept + np.outer(t, quadratic_firstorder_dir) + np.outer(t*t, quadratic_secondorder_dir)



def compute_t_for_3d_point(
    P, 
    model_type, 
    config,
    z_intercept,
    linear_dir=None,
    quadratic_firstorder_dir=None, 
    quadratic_secondorder_dir=None,
    t_bounds=None
):
    '''
    Compute the scalar lane parameter t corresponding to the projection of a 3D point 
    onto a linear or quadratic lane model.

    Args:
        P (3,) np array: 3D point to project.
        model_type (str): the used fitting model, either "linear" or "quadratic".
        config (dict): Configuration file.
        z_intercept (3,) np array: 3D point on plane for the y-intercept.

        Either:
            1) Linear:
                linear_dir (3,) np array: Unit direction vector along the fitted line.
                quadratic_firstorder_dir, quadratic_secondorder_dir = None
            2) Quadratic:
                linear_dir = None             
                quadratic_firstorder_dir (3,) np array: Unit direction vector for first-order change along the fitted line.
                quadratic_secondorder_dir (3,) np array: Unit direction vector for second-order change along the fitted line.
        
        t_bounds (tuple[float, float], optional): Lower and upper bounds for the optimizer search range on t.
            Defaults to config file settings.

    Returns:
        Parametric distance along lane line (float).
    '''
    P = np.asarray(P)
    p0 = z_intercept
    if model_type == "linear":
        # use linear_intercept and linear_dir
        a = linear_dir
        t = float(np.dot(P - p0, a))
        return t
    elif model_type == "quadratic":
        a = quadratic_firstorder_dir
        b = quadratic_secondorder_dir
        def f(t):
            X = p0 + a * t + b * (t*t)
            return np.sum((X - P)**2)
        # initial guess from projection onto a
        t0 = float(np.dot(P - p0, a))
        if t_bounds is None:
            hi = config["projection_mapping"]["curve_optimizer_fallback_search_range"]
            lo = -hi
        else:
            lo, hi = t_bounds
        short_range = config["projection_mapping"]["curve_optimizer_short_search_range"]
        lo_try = max(lo, t0 - short_range)
        hi_try = min(hi, t0 + short_range)
        if lo_try >= hi_try:
            lo_try, hi_try = lo, hi
        res = minimize_scalar(
            f, 
            bounds=(lo_try, hi_try), 
            method='bounded', 
            options={'xatol':float(config["projection_mapping"]["curve_optimizer_precise_xatol"])}
        )
        if not res.success:
            res = minimize_scalar(
                f, 
                bounds=(lo, hi), 
                method='bounded', 
                options={'xatol':float(config["projection_mapping"]["curve_optimizer_fallback_xatol"])}
            )
        return float(res.x)
    else:
        if log_specifics:
            logger.error("Unsupported model_type: must be 'linear' or 'quadratic'")
        else:
            print("Unsupported model_type: must be 'linear' or 'quadratic'")
        sys.exit(1)



def learn_pixel_to_t_mapping_for_lane(
    s1_points,               
    model_type,             
    config,
    z_intercept,            
    img_height, fx, fy, cx, cy, distortion=None,
    linear_dir=None,         
    quadratic_firstorder_dir=None,
    quadratic_secondorder_dir=None
):
    '''
    Learn a mapping between pixel-space arc length (s_pixel) and lane parameter t 
    for a given lane, based on known 2D-3D correspondences.

    Args:
        s1_points (N,3) np array: points (u, v, d) to learn mapping.
        model_type (str): the used fitting model, either "linear" or "quadratic".
        config (dict): Configuration file.
        z_intercept (3,) np array: 3D point on plane for the y-intercept.
        img_height (float): Height of image (px).
        fx, fy, cx, cy (float): Intrinsic parameters.
        distortion (5,) optional np array: [k1, k2, p1, p2, k3] distortion coefficients.

        Either:
            1) Linear:
                linear_dir (3,) np array: Unit direction vector along the fitted line.
                quadratic_firstorder_dir, quadratic_secondorder_dir = None
            2) Quadratic:
                linear_dir = None             
                quadratic_firstorder_dir (3,) np array: Unit direction vector for first-order change along the fitted line.
                quadratic_secondorder_dir (3,) np array: Unit direction vector for second-order change along the fitted line.

    Returns:
        tuple: (
            PchipInterpolator (function), pixel arc-length s_pixel → lane parameter t,
            Sorted 2D pixel polyline (u,v) along increasing t (N,2) np array,
            Cumulative 2D pixel arc-lengths (N,) np array,
            Corresponding lane parameters (N,) np array,
            Corresponding 3D points along the lane (N,3) np array
        )
    '''
    # Convert to 2D to 3D
    u = s1_points[:,0]; v = s1_points[:,1]; d = s1_points[:,2]
    P3 = project_pixel_to_depth(u, img_height-v, d, fx, fy, cx, cy, distortion)

    # Compute t_i for each 3D point
    N = P3.shape[0]
    t_vals = np.zeros(N, dtype=float)
    max_range = np.linalg.norm(P3 - z_intercept, axis=1).max()
    range_mult = config["projection_mapping"]["curve_optimizer_full_search_range_mult"]
    t_bounds = (-max_range*range_mult - 1.0, max_range*range_mult + 1.0)
    for i in range(N):
        P = P3[i]
        if model_type == "linear":
            t_vals[i] = compute_t_for_3d_point(
                P, model_type, config, z_intercept,
                linear_dir=linear_dir,
                t_bounds=t_bounds
            )
        else:
            t_vals[i] = compute_t_for_3d_point(
                P, model_type, config, z_intercept,
                quadratic_firstorder_dir=quadratic_firstorder_dir,
                quadratic_secondorder_dir=quadratic_secondorder_dir,
                t_bounds=t_bounds
            )

    # Sort by t to get along-lane order
    order = np.argsort(t_vals)
    t_sorted = t_vals[order]
    s1_sorted = s1_points[order]
    P3_sorted = P3[order]
    pixel2d_sorted = s1_sorted[:, :2]

    # compute s_pixel (cumulative pixel arc-length along the sorted S1 polyline)
    s_pixel_sorted = cumulative_lengths_2d(pixel2d_sorted)

    # Fit robust 1D mapping s_pixel to t
    mask_valid = np.isfinite(s_pixel_sorted) & np.isfinite(t_sorted)
    s_fit = s_pixel_sorted[mask_valid]
    t_fit = t_sorted[mask_valid]
    if len(s_fit) < 3:      # min number points needed for PCHIP
        if log_specifics:
            logger.error("Need >= 3 S1 points to learn mapping for this lane.")
        else:
            print("Need >= 3 S1 points to learn mapping for this lane.")
        sys.exit(1)
    # quick linear resid-based outlier removal (MAD)
    A = np.vstack([s_fit, np.ones_like(s_fit)]).T
    lin_coef, lin_intercept = np.linalg.lstsq(A, t_fit, rcond=None)[0]
    resid = np.abs(t_fit - (lin_coef * s_fit + lin_intercept))
    med = np.median(resid)
    mad = np.median(np.abs(resid - med)) + 1e-9
    outlier_mask = resid > (mad * config["projection_mapping"]["median_absolute_deviation_multiplier"])
    keep_mask = ~outlier_mask
    if np.sum(keep_mask) < max(3, int(len(keep_mask) * config["projection_mapping"]["percent_min_points_remaining"])):
        keep_mask = np.ones_like(keep_mask, dtype=bool)
    s_fit2 = s_fit[keep_mask]
    t_fit2 = t_fit[keep_mask]

    order2 = np.argsort(s_fit2)
    s_fit_sorted = s_fit2[order2]
    t_fit_sorted = t_fit2[order2]
    # jitter duplicates if any
    ds = np.diff(s_fit_sorted)
    if np.any(ds == 0):
        eps = 1e-8
        for i in range(1, len(s_fit_sorted)):
            if s_fit_sorted[i] <= s_fit_sorted[i-1]:
                s_fit_sorted[i] = s_fit_sorted[i-1] + eps

    interp_s_to_t = PchipInterpolator(s_fit_sorted, t_fit_sorted, extrapolate=True)

    return interp_s_to_t, pixel2d_sorted, s_pixel_sorted, t_sorted, P3_sorted



def predict_3d_for_s2_points(
    s2_points,             
    interp_s_to_t,           
    pixel2d_polyline,       
    model_type,
    z_intercept,
    linear_dir=None,
    quadratic_firstorder_dir=None, quadratic_secondorder_dir=None
):
    '''
    Predict 3D lane coordinates for new 2D pixel points using a previously learned
    pixel-to-lane mapping and 3D lane model.

    Args:
        s2_points (N,2) np array: points (u, v) to predict 3D position.
        interp_s_to_t (function): PchipInterpolator mapping pixel arc-length → lane parameter t
            (see learn_pixel_to_t_mapping_for_lane()).
        pixel2d_polyline (N,2) np array:  Sorted 2D pixel polyline (u,v) along increasing t
        model_type (str): the used fitting model, either "linear" or "quadratic".
        z_intercept (3,) np array: 3D point on plane for the y-intercept.

        Either:
            1) Linear:
                linear_dir (3,) np array: Unit direction vector along the fitted line.
                quadratic_firstorder_dir, quadratic_secondorder_dir = None
            2) Quadratic:
                linear_dir = None             
                quadratic_firstorder_dir (3,) np array: Unit direction vector for first-order change along the fitted line.
                quadratic_secondorder_dir (3,) np array: Unit direction vector for second-order change along the fitted line.

    Returns:
        tuple: (
            Predicted 3D coordinates of each pixel (N,3) pd array,
            Corresponding lane parameter values along curve (N,) pd array,
            Cumulative arc-length distances (px) along 2D polyline (N,) pd array
        )
    '''
    M = s2_points.shape[0]
    s_new = np.zeros(M, dtype=float)
    for i, pt in enumerate(s2_points):
        s_new[i], _, _, _, _ = project_point_to_2d_polyline(pt, pixel2d_polyline)

    global minty_fresh
    t_new = interp_s_to_t(s_new)
    
    if model_type == "linear":
        X3 = lane_point_linear_model(z_intercept, linear_dir, t_new)
    else:
        X3 = lane_point_quadratic_model(z_intercept, quadratic_firstorder_dir, quadratic_secondorder_dir, t_new)
    return X3, t_new, s_new



def project_point_linear(P, z_intercept, linear_dir):
    """
    Project a 3D point onto a linear lane model.

    Args:
        P (3,) np array: The 3D point to project.
        z_intercept (3,) np array: 3D point on plane for the y-intercept.
        linear_dir (3,) np array: Unit direction vector of the lane.

    Returns: 
        tuple: (
            t (float): Scalar lane coordinate along the line,
            P_proj (3,) np array: The projected 3D point lying on the line
        )
    """

    t = np.dot(P - z_intercept, linear_dir)
    P_proj = z_intercept + t * linear_dir
    return t, P_proj



def project_point_quadratic(P, z_intercept, quadratic_firstorder_dir, quadratic_secondorder_dir):
    """
    Project a 3D point onto a quadratic lane model.

    Args:
        P (3,) np array: The 3D point to project.
        z_intercept (3,) np array: 3D point on plane for the y-intercept.
        quadratic_firstorder_dir (3,) np array: Unit direction vector for first-order change along the fitted line.
        quadratic_secondorder_dir (3,) np array: Unit direction vector for second-order change along the fitted line.

    Returns: 
        tuple: (
            t (float): Scalar lane coordinate along the quadratic curve,
            P_proj (3,) np array: The projected 3D point lying on the quadratic curve
        )
    """
    r = z_intercept - P

    # Coefficients of derivative cubic
    A = 2 * np.dot(quadratic_secondorder_dir, quadratic_secondorder_dir)
    B = 3 * np.dot(quadratic_firstorder_dir, quadratic_secondorder_dir)
    C = 2 * np.dot(quadratic_firstorder_dir, quadratic_firstorder_dir) + np.dot(r, quadratic_secondorder_dir)
    D = np.dot(r, quadratic_firstorder_dir)

    # Solve cubic: A t³ + B t² + C t + D = 0
    coeffs = [A, B, C, D]

    roots = np.roots(coeffs)
    real_roots = roots[np.isreal(roots)].real

    # Evaluate error for all real roots
    def error(t):
        PT = z_intercept + t * quadratic_firstorder_dir + t * t * quadratic_secondorder_dir
        return np.sum((P - PT)**2)

    t_best = min(real_roots, key=error)
    P_proj = z_intercept + t_best * quadratic_firstorder_dir + t_best * t_best * quadratic_secondorder_dir

    return t_best, P_proj



def project_points(
    points, 
    model_type,
    z_intercept,
    linear_dir=None,
    quadratic_firstorder_dir=None, 
    quadratic_secondorder_dir=None
):
    """
    Project a set of 3D points onto a linear or quadratic lane model.

    Args:
        points (N,3) np array: Set of 3D points to project.
        model_type (str): the used fitting model, either "linear" or "quadratic".
        config (dict): Configuration file.
        z_intercept (3,) np array: 3D point on plane for the y-intercept.

        Either:
            1) Linear:
                linear_dir (3,) np array: Unit direction vector along the fitted line.
                quadratic_firstorder_dir, quadratic_secondorder_dir = None
            2) Quadratic:
                linear_dir = None             
                quadratic_firstorder_dir (3,) np array: Unit direction vector for first-order change along the fitted line.
                quadratic_secondorder_dir (3,) np array: Unit direction vector for second-order change along the fitted line.

    Returns:
        t_list (N,) np array: Scalar lane coordinates for each point.
        proj_list (N,3) np array: Projected points on the lane model.
    """
    t_list = []
    proj_list = []

    if model_type == "linear":
        for P in points:
            t, Pp = project_point_linear(P, z_intercept, linear_dir)
            t_list.append(t)
            proj_list.append(Pp)
    else:  # quadratic
        for P in points:
            t, Pp = project_point_quadratic(P, z_intercept, quadratic_firstorder_dir, quadratic_secondorder_dir)
            t_list.append(t)
            proj_list.append(Pp)

    return np.array(t_list), np.array(proj_list)



def correct_z_mismatched_points(
    X3_pred,              
    intrinsic_points,      
    config,
    model_type,
    z_intercept,
    linear_dir=None,
    quadratic_firstorder_dir=None, 
    quadratic_secondorder_dir=None
):
    """
    Correct inconsistent predicted 3D lane points using intrinsic depth references
    and parametric projection where needed.

    Args:
        X3_pred (N,3) np array: Predicted 3D points.
        intrinsic_points (N,3) np array: Reference 3D points derived from intrinsic projection.
        config (dict): Configuration file.
        model_type (str): the used fitting model, either "linear" or "quadratic".
        config (dict): Configuration file.
        z_intercept (3,) np array: 3D point on plane for the y-intercept.

        Either:
            1) Linear:
                linear_dir (3,) np array: Unit direction vector along the fitted line.
                quadratic_firstorder_dir, quadratic_secondorder_dir = None
            2) Quadratic:
                linear_dir = None             
                quadratic_firstorder_dir (3,) np array: Unit direction vector for first-order change along the fitted line.
                quadratic_secondorder_dir (3,) np array: Unit direction vector for second-order change along the fitted line.

    Returns:
        corrected_points (N,3) np array: Updated 3D points with potential replaced mismatched points.
    """
    X3_pred = np.asarray(X3_pred)
    intrinsic_points = np.asarray(intrinsic_points)
    corrected_points = X3_pred.copy()

    # Check z-range mismatch
    max_deviation = config["depth_map"]["max_deviation_from_intr"]
    intrinsic_z = intrinsic_points[:, 2]
    pred_z = X3_pred[:, 2]
    lower = intrinsic_z * (1 - max_deviation)
    upper = intrinsic_z * (1 + max_deviation)
    mask_mismatch = (pred_z < lower) | (pred_z > upper)

    if not np.any(mask_mismatch):
        return corrected_points  # all points valid

    # Process mismatched points
    for _ in np.where(mask_mismatch)[0]:
        idx = np.where(mask_mismatch)[0]
        pts_to_project = intrinsic_points[idx]
        _, projected_subset = project_points(
            pts_to_project, 
            model_type, 
            z_intercept,
            linear_dir,
            quadratic_firstorder_dir, 
            quadratic_secondorder_dir
        )
        corrected_points[idx] = projected_subset

    return corrected_points



def save_pointcloud_sequence(arrays, base_folder, seq_id, sub_folder, file_id):
    '''
    Save a list of (N,3) np arrays of point clouds to .bin files in structured folders.

    Directory structure:
        base_folder/seq_id/sub_folder/<file_id>_<idx>.bin

    Args:
        arrays: list of (N,3) np arrays of point clouds (of individual lanes) to save.
        base_folder, seq_id, sub_folder, file_id (str or int) build the directory structure.
    '''
    # Ensure directory exists
    save_dir = os.path.join(base_folder, str(seq_id), sub_folder)
    os.makedirs(save_dir, exist_ok=True)

    # Loop and write each point cloud
    for idx, points in enumerate(arrays):
        if not isinstance(points, np.ndarray):
            if log_specifics:
                logger.error(f"Item {idx} is not a numpy array.")
            else:
                print(f"Item {idx} is not a numpy array.")
            sys.exit(1)
        if points.ndim != 2 or points.shape[1] != 3:
            if log_specifics:
                logger.error(f"Item {idx} must be of shape (N,3), got {points.shape}")
            else:
                print(f"Item {idx} must be of shape (N,3), got {points.shape}")
            sys.exit(1)

        file_path = os.path.join(save_dir, f"{file_id}_{idx}.bin")
        points.astype(np.float32).tofile(file_path)


### Main Function

In [ ]:
def run_cl_intrinsic(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files):
    '''
    This function implements a pure-intrinsic approach: it projects 2D lane pixels
    into 3D space using only the camera intrinsics, image size, distortion parameters,
    and a fixed sensor height, without relying on LiDAR point correspondences or
    ground plane estimation. Each lane in a LiDAR file produces one 3D output.

    Run a full processing pipeline for a single solution on a set of LiDAR files.
    This function iterates over each LiDAR file, performs any preprocessing steps, 
    applies the main processing routine to generate 3D outputs, and saves the results. 

    Args:
        once_calibration_path (str): Path to the calibration file used for processing.
        lane_detections_folder_path (str): Path to the folder containing the input 2D detection files.
        seq_id (str or int): The run id / sequence id. 
        lidar_files (list of Path): List of LiDAR file paths to process.

    Outputs:
        The processed 3D results are saved to disk. Each lane is saved as a separate 
        .bin file, named as <file_id>_<lane_idx>.bin, under the output path specified 
        in the configuration file for this solution. The first lane in a file has index 0,
        the next lane index 1, and so on.

    Returns:
        total_lane_count (int): the total count of lanes (attempted to) process.
    '''
    (
        _,
        _,
        fx, fy, cx, cy,
        img_width, img_height,
        distortion
    ) = load_calibration(once_calibration_path)
    
    total_files = len(lidar_files)
    total_lane_count = 0

    for idx, once_lidar_path in enumerate(tqdm(lidar_files, desc=f"Processing cl intrinsic LiDAR files for {seq_id}"), start=1):
        ########## Run Check ##########
        file_id = str(Path(once_lidar_path).stem)

        if once_config["runtime"]["file_id"] is not None:
            if file_id != once_config["runtime"]["file_id"]:
                continue
        elif not once_config["runtime"]["override_existing_output"] and Path(os.path.join(once_config["output"]["base_path"], seq_id, once_config["output"]["cl_intrinsic_path"], file_id + "_0.bin")).exists():
            logger.info(f"File [{idx}/{total_files}]: File already processed: {file_id}")
            continue
        else:
            logger.info(f"File [{idx}/{total_files}]: Processing:             {file_id}")
        ###############################



        ########## Preprocessing ##########

        ##### Load Input 2D Lanes #####
        lanes = load_2d_lanes(os.path.join(lane_detections_folder_path, file_id + ".txt"), once_config, img_width, img_height)
        if lanes is None or len(lanes) == 0:
            logger.error(f"No lanes are found.")
            continue
        ###################################



        ########## Main Processing ##########
        classic_pure_intrinsic = []
        for lane_pixels in lanes:
            lane_x = lane_pixels[:, 0]
            lane_y = lane_pixels[:, 1]
            classic_pure_intrinsic.append(pure_intrinsic_projection(lane_x, img_height-lane_y, fx, fy, cx, cy, once_config, distortion, once_config["ground_plane_ransac"]["camera_height"]))
            total_lane_count += 1
        #####################################



        ########## File Saving ##########
        save_pointcloud_sequence( 
            classic_pure_intrinsic,
            once_config["output"]["base_path"],
            seq_id,
            once_config["output"]["cl_intrinsic_path"],
            file_id
        )
        #################################
    
    return total_lane_count


In [ ]:
def run_cl_intrinsic_groundplane(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files):
    '''
    This function implements a ground-plane projection approach: it projects 2D lane pixels
    into 3D space by first estimating the ground plane from the LiDAR points aligned in the
    camera frame. Each pixel is then projected onto this estimated plane, rather than using
    a fixed camera height. This approach accounts for the actual ground geometry in the scene,
    producing one 3D output per lane in each LiDAR file.

    Run a full processing pipeline for a single solution on a set of LiDAR files.
    This function iterates over each LiDAR file, performs any preprocessing steps, 
    applies the main processing routine to generate 3D outputs, and saves the results. 

    Args:
        once_calibration_path (str): Path to the calibration file used for processing.
        lane_detections_folder_path (str): Path to the folder containing the input 2D detection files.
        seq_id (str or int): The run id / sequence id. 
        lidar_files (list of Path): List of LiDAR file paths to process.

    Outputs:
        The processed 3D results are saved to disk. Each lane is saved as a separate 
        .bin file, named as <file_id>_<lane_idx>.bin, under the output path specified 
        in the configuration file for this solution. The first lane in a file has index 0,
        the next lane index 1, and so on.

    Returns:
        total_lane_count (int): the total count of lanes (attempted to) process.
    '''
    (
        T_camera_lidar,
        _,
        fx, fy, cx, cy,
        img_width, img_height,
        distortion
    ) = load_calibration(once_calibration_path)
    
    total_files = len(lidar_files)
    total_lane_count = 0

    for idx, once_lidar_path in enumerate(tqdm(lidar_files, desc=f"Processing cl intrinsic ground plane LiDAR files for {seq_id}"), start=1):
        ########## Run Check ##########
        file_id = str(Path(once_lidar_path).stem)

        if once_config["runtime"]["file_id"] is not None:
            if file_id != once_config["runtime"]["file_id"]:
                continue
        elif not once_config["runtime"]["override_existing_output"] and Path(os.path.join(once_config["output"]["base_path"], seq_id, once_config["output"]["cl_intrinsic_ground_path"], file_id + "_0.bin")).exists():
            logger.info(f"File [{idx}/{total_files}]: File already processed: {file_id}")
            continue
        else:
            logger.info(f"File [{idx}/{total_files}]: Processing:             {file_id}")
        ###############################



        ########## Preprocessing ##########

        ##### Load Input 2D Lanes #####
        lanes = load_2d_lanes(os.path.join(lane_detections_folder_path, file_id + ".txt"), once_config, img_width, img_height)
        if lanes is None or len(lanes) == 0:
            logger.error(f"No lanes are found.")
            continue


        ##### Calculate Ground Plane #####
        lidar_points, _, _, _ = get_lidar_points(once_lidar_path)
        
        # Transform points from lidar to camera frame
        points_lidar_hom = np.hstack((lidar_points, np.ones((lidar_points.shape[0], 1))))
        points_lidar_camera = (T_camera_lidar @ points_lidar_hom.T).T[:,:3] # lidar -> camera
        
        # Get ground plane
        prefiltered_points = prefilter_ground_plane(
            points_lidar_camera, 
            -once_config["ground_plane_ransac"]["camera_height"],
            once_config["ground_plane_ransac"]["prefilter_band_half_width"],
            vertical_axis=1
        )
        plane_normal, plane_offset, _ = estimate_ground_plane(
            prefiltered_points,
            once_config,
            vertical_axis=0
        )
        ###################################



        ########## Main Processing ##########
        classic_intrinsic_ground_plane = []
        for lane_pixels in lanes:
            lane_x = lane_pixels[:, 0]
            lane_y = lane_pixels[:, 1]
            intrinsic_points = pure_intrinsic_projection(lane_x, img_height - lane_y, fx, fy, cx, cy, once_config, distortion, None, plane_normal, plane_offset)
            classic_intrinsic_ground_plane.append(intrinsic_points)
            total_lane_count += 1
        #####################################



        ########## File Saving ##########
        save_pointcloud_sequence( 
            classic_intrinsic_ground_plane,
            once_config["output"]["base_path"],
            seq_id,
            once_config["output"]["cl_intrinsic_ground_path"],
            file_id
        )
        #################################
    
    return total_lane_count


In [ ]:
def run_cl_depth(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files):
    '''
    This function implements a LiDAR depth-based matching approach: it projects 2D lane pixels
    into 3D space by first aligning LiDAR points to the camera frame and estimating the ground plane.
    Only ground points within the camera's field of view are retained. These points are organized
    into spatial bins corresponding to their 2D projections. For each lane pixel, the algorithm
    searches the bins to find the nearest LiDAR point, expanding to neighboring bins if necessary.
    The matched 3D point from LiDAR then becomes the 3D position of the pixel. This approach
    directly leverages LiDAR depth to provide per-pixel 3D correspondences.

    Run a full processing pipeline for a single solution on a set of LiDAR files.
    This function iterates over each LiDAR file, performs any preprocessing steps, 
    applies the main processing routine to generate 3D outputs, and saves the results. 

    Args:
        once_calibration_path (str): Path to the calibration file used for processing.
        lane_detections_folder_path (str): Path to the folder containing the input 2D detection files.
        seq_id (str or int): The run id / sequence id. 
        lidar_files (list of Path): List of LiDAR file paths to process.

    Outputs:
        The processed 3D results are saved to disk. Each lane is saved as a separate 
        .bin file, named as <file_id>_<lane_idx>.bin, under the output path specified 
        in the configuration file for this solution. The first lane in a file has index 0,
        the next lane index 1, and so on.

    Returns:
        total_lane_count (int): the total count of lanes (attempted to) process.
    '''
    (
        T_camera_lidar,
        _,
        fx, fy, cx, cy,
        img_width, img_height,
        distortion
    ) = load_calibration(once_calibration_path)
    
    total_files = len(lidar_files)
    total_lane_count = 0

    for idx, once_lidar_path in enumerate(tqdm(lidar_files, desc=f"Processing cl depth LiDAR files for {seq_id}"), start=1):
        ########## Run Check ##########
        file_id = str(Path(once_lidar_path).stem)

        if once_config["runtime"]["file_id"] is not None:
            if file_id != once_config["runtime"]["file_id"]:
                continue
        elif not once_config["runtime"]["override_existing_output"] and Path(os.path.join(once_config["output"]["base_path"], seq_id, once_config["output"]["cl_depth_path"], file_id + "_0.bin")).exists():
            logger.info(f"File [{idx}/{total_files}]: File already processed: {file_id}")
            continue
        else:
            logger.info(f"File [{idx}/{total_files}]: Processing:             {file_id}")
        ###############################



        ########## Preprocessing ##########

        ##### Load Input 2D Lanes #####
        lanes = load_2d_lanes(os.path.join(lane_detections_folder_path, file_id + ".txt"), once_config, img_width, img_height)
        if lanes is None or len(lanes) == 0:
            logger.error(f"No lanes are found.")
            continue


        ##### Calculate Ground Plane #####
        lidar_points, _, _, _ = get_lidar_points(once_lidar_path)
        
        # Transform points from lidar to camera frame
        points_lidar_hom = np.hstack((lidar_points, np.ones((lidar_points.shape[0], 1))))
        points_lidar_camera = (T_camera_lidar @ points_lidar_hom.T).T[:,:3] # lidar -> camera

        # Get ground plane
        prefiltered_points = prefilter_ground_plane(
            points_lidar_camera, 
            -once_config["ground_plane_ransac"]["camera_height"],
            once_config["ground_plane_ransac"]["prefilter_band_half_width"],
            vertical_axis=1
        )
        _, _, ground_plane_mask = estimate_ground_plane(
            prefiltered_points,
            once_config,
            vertical_axis=0
        )
        ground_points = prefiltered_points[ground_plane_mask]

        # Filter points into FOV
        u_ground, v_ground, d_ground, sizes_sorted, ground_points_fov = filter_and_clip_points(
            ground_points,
            fx, fy, cx, cy, img_width, img_height,
            once_config,
            distortion=distortion
        )

        # Create depth based search
        dpi = once_config["depth_map"]["dpi"]
        RADIUS_SCALE = dpi / (72 * np.sqrt(np.pi))      # Convert dpi from point to px, set to radius
        cell_size = once_config["depth_map"]["lidar_point_size_max"] * RADIUS_SCALE
        r = (sizes_sorted * RADIUS_SCALE)
        spatial_bins = create_spatial_bins(u_ground, v_ground, r, d_ground, ground_points_fov, cell_size)
        ###################################



        ########## Main Processing ##########
        classic_pure_depth = []
        for lane_pixels in lanes:
            lane_x = lane_pixels[:, 0]
            lane_y = lane_pixels[:, 1]

            matched_points = []
            for i in range(len(lane_x)):
                _, _, _, pt = query_nearest_depth_from_bins(spatial_bins, lane_x[i], lane_y[i], bin_search_range=None) #search infinitely
                matched_points.append(pt)
            classic_pure_depth.append(np.array(matched_points))
            
            total_lane_count += 1
        #####################################



        ########## File Saving ##########
        save_pointcloud_sequence( 
            classic_pure_depth,
            once_config["output"]["base_path"],
            seq_id,
            once_config["output"]["cl_depth_path"],
            file_id
        )
        #################################


    return total_lane_count


In [ ]:
def run_cl_intrinsic_depth(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files):
    '''
    This function implements a fused intrinsic + LiDAR matching approach: it projects 2D lane pixels
    into 3D space by first aligning LiDAR points to the camera frame and estimating the ground plane.
    Only ground points within the camera's field of view are retained and organized into spatial bins.
    For each lane pixel, the nearest LiDAR point is found in the bins, and its depth along the
    camera viewing ray is extracted. The 3D point is then computed using the original pixel coordinates
    with this matched depth, combining the accuracy of LiDAR depth with the stability of intrinsic-based
    projection. Each lane in a LiDAR file produces one 3D output.

    Run a full processing pipeline for a single solution on a set of LiDAR files.
    This function iterates over each LiDAR file, performs any preprocessing steps, 
    applies the main processing routine to generate 3D outputs, and saves the results. 

    Args:
        once_calibration_path (str): Path to the calibration file used for processing.
        lane_detections_folder_path (str): Path to the folder containing the input 2D detection files.
        seq_id (str or int): The run id / sequence id. 
        lidar_files (list of Path): List of LiDAR file paths to process.

    Outputs:
        The processed 3D results are saved to disk. Each lane is saved as a separate 
        .bin file, named as <file_id>_<lane_idx>.bin, under the output path specified 
        in the configuration file for this solution. The first lane in a file has index 0,
        the next lane index 1, and so on.

    Returns:
        total_lane_count (int): the total count of lanes (attempted to) process.
    '''
    (
        T_camera_lidar,
        _,
        fx, fy, cx, cy,
        img_width, img_height,
        distortion
    ) = load_calibration(once_calibration_path)
    
    total_files = len(lidar_files)
    total_lane_count = 0

    for idx, once_lidar_path in enumerate(tqdm(lidar_files, desc=f"Processing cl intrinsic + depth LiDAR files for {seq_id}"), start=1):
        ########## Run Check ##########
        file_id = str(Path(once_lidar_path).stem)

        if once_config["runtime"]["file_id"] is not None:
            if file_id != once_config["runtime"]["file_id"]:
                continue
        elif not once_config["runtime"]["override_existing_output"] and Path(os.path.join(once_config["output"]["base_path"], seq_id, once_config["output"]["cl_intrinsic_depth_path"], file_id + "_0.bin")).exists():
            logger.info(f"File [{idx}/{total_files}]: File already processed: {file_id}")
            continue
        else:
            logger.info(f"File [{idx}/{total_files}]: Processing:             {file_id}")
        ###############################



        ########## Preprocessing ##########

        ##### Load Input 2D Lanes #####
        lanes = load_2d_lanes(os.path.join(lane_detections_folder_path, file_id + ".txt"), once_config, img_width, img_height)
        if lanes is None or len(lanes) == 0:
            logger.error(f"No lanes are found.")
            continue


        ##### Calculate Ground Plane #####
        lidar_points, _, _, _ = get_lidar_points(once_lidar_path)
        
        # Transform points from lidar to camera frame
        points_lidar_hom = np.hstack((lidar_points, np.ones((lidar_points.shape[0], 1))))
        points_lidar_camera = (T_camera_lidar @ points_lidar_hom.T).T[:,:3] # lidar -> camera

        # Get ground plane
        prefiltered_points = prefilter_ground_plane(
            points_lidar_camera, 
            -once_config["ground_plane_ransac"]["camera_height"],
            once_config["ground_plane_ransac"]["prefilter_band_half_width"],
            vertical_axis=1
        )
        _, _, ground_plane_mask = estimate_ground_plane(
            prefiltered_points,
            once_config,
            vertical_axis=0
        )
        ground_points = prefiltered_points[ground_plane_mask]

        # Filter points into FOV
        u_ground, v_ground, d_ground, sizes_sorted, ground_points_fov = filter_and_clip_points(
            ground_points,
            fx, fy, cx, cy, img_width, img_height,
            once_config,
            distortion=distortion
        )

        # Create depth based search
        dpi = once_config["depth_map"]["dpi"]
        RADIUS_SCALE = dpi / (72 * np.sqrt(np.pi))      # Convert dpi from point to px, set to radius
        cell_size = once_config["depth_map"]["lidar_point_size_max"] * RADIUS_SCALE
        r = (sizes_sorted * RADIUS_SCALE)
        spatial_bins = create_spatial_bins(u_ground, v_ground, r, d_ground, ground_points_fov, cell_size)
        ###################################



        ########## Main Processing ##########
        classic_combined_intrinsic_depth = []
        for lane_pixels in lanes:
            lane_x = lane_pixels[:, 0]
            lane_y = lane_pixels[:, 1]

            udv_point = []
            for i in range(len(lane_x)):
                _, _, depth, _ = query_nearest_depth_from_bins(spatial_bins, lane_x[i], lane_y[i], bin_search_range=None) #search infinitely
                udv_point.append(np.array([lane_x[i], lane_y[i], depth]))
            udv_point = np.array(udv_point)
            classic_combined_intrinsic_depth.append(project_pixel_to_depth(udv_point[:,0], img_height - udv_point[:,1], udv_point[:,2], fx, fy, cx, cy, distortion))
            
            total_lane_count += 1
        #####################################



        ########## File Saving ##########
        save_pointcloud_sequence( 
            classic_combined_intrinsic_depth,
            once_config["output"]["base_path"],
            seq_id,
            once_config["output"]["cl_intrinsic_depth_path"],
            file_id
        )
        #################################
    
    return total_lane_count


In [ ]:
def run_thesis_solution(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files):
    '''
    This function implements a custom lane reconstruction approach that fuses intrinsic projection,
    LiDAR depth, and geometric line fitting to produce accurate 3D lane points. LiDAR points are first
    aligned to the camera and filtered to retain only ground points within the camera's field of view.
    A ground plane is estimated, and the 2D lane anchors are projected onto it using a pure-intrinsic
    method. A linear or quadratic 3D line is fit through these projected points to represent the lane.
    
    For each set of consecutive lane anchors, all pixels in between are evaluated: the nearest LiDAR
    point is searched using spatial bins, and the corresponding 2D match is projected onto the fitted
    lane line. This produces accurate depth and 3D correspondences for each pixel. End segments of the
    lane are processed with a larger search radius to avoid over-extrapolation.
    
    A mapping between pixel-space arc length and lane distance is then learned (using a PCHIP
    interpolator), allowing the original 3D lane anchors to be projected along the fitted line according
    to this mapping. Each lane in a LiDAR file produces one set of 3D output points.

    Run a full processing pipeline for a single solution on a set of LiDAR files.
    This function iterates over each LiDAR file, performs any preprocessing steps, 
    applies the main processing routine to generate 3D outputs, and saves the results. 

    Args:
        once_calibration_path (str): Path to the calibration file used for processing.
        lane_detections_folder_path (str): Path to the folder containing the input 2D detection files.
        seq_id (str or int): The run id / sequence id. 
        lidar_files (list of Path): List of LiDAR file paths to process.

    Outputs:
        The processed 3D results are saved to disk. Each lane is saved as a separate 
        .bin file, named as <file_id>_<lane_idx>.bin, under the output path specified 
        in the configuration file for this solution. The first lane in a file has index 0,
        the next lane index 1, and so on.

    Returns:
        total_lane_count (int): the total count of lanes (attempted to) process.
    '''
    (
        T_camera_lidar,
        _,
        fx, fy, cx, cy,
        img_width, img_height,
        distortion
    ) = load_calibration(once_calibration_path)
    
    total_files = len(lidar_files)
    total_lane_count = 0

    dpi = once_config["depth_map"]["dpi"]
    RADIUS_SCALE = dpi / (72 * np.sqrt(np.pi))      # Convert dpi from point to px, set to radius
    cell_size = once_config["depth_map"]["lidar_point_size_max"] * RADIUS_SCALE
    max_search_range = once_config["depth_map"]["unmatched_max_search_range"]
    logger.info(f"Max search range: {cell_size * 2 * max_search_range}")


    for idx, once_lidar_path in enumerate(tqdm(lidar_files, desc=f"Processing thesis solution LiDAR files for {seq_id}"), start=1):
        ########## Run Check ##########
        file_id = str(Path(once_lidar_path).stem)

        if once_config["runtime"]["file_id"] is not None:
            if file_id != once_config["runtime"]["file_id"]:
                continue
        elif not once_config["runtime"]["override_existing_output"] and Path(os.path.join(once_config["output"]["base_path"], seq_id, once_config["output"]["thesis_solution_path"], file_id + "_0.bin")).exists():
            logger.info(f"File [{idx}/{total_files}]: File already processed: {file_id}")
            continue
        else:
            logger.info(f"File [{idx}/{total_files}]: Processing:             {file_id}")
        ###############################



        ########## Preprocessing ##########

        ##### Load Input 2D Lanes #####
        lanes = load_2d_lanes(os.path.join(lane_detections_folder_path, file_id + ".txt"), once_config, img_width, img_height)
        if lanes is None or len(lanes) == 0:
            logger.error(f"No lanes are found.")
            continue


        ##### Calculate Ground Plane #####
        lidar_points, _, _, _ = get_lidar_points(once_lidar_path)
        
        # Transform points from lidar to camera frame
        points_lidar_hom = np.hstack((lidar_points, np.ones((lidar_points.shape[0], 1))))
        points_lidar_camera = (T_camera_lidar @ points_lidar_hom.T).T[:,:3] # lidar -> camera

        # Get ground plane
        prefiltered_points = prefilter_ground_plane(
            points_lidar_camera, 
            -once_config["ground_plane_ransac"]["camera_height"],
            once_config["ground_plane_ransac"]["prefilter_band_half_width"],
            vertical_axis=1
        )
        plane_normal, plane_offset, ground_plane_mask = estimate_ground_plane(
            prefiltered_points,
            once_config,
            vertical_axis=0
        )
        ground_points = prefiltered_points[ground_plane_mask]

        # Filter points into FOV
        u_ground, v_ground, d_ground, sizes_sorted, ground_points_fov = filter_and_clip_points(
            ground_points,
            fx, fy, cx, cy, img_width, img_height,
            once_config,
            distortion=distortion
        )

        # Create depth based search
        r = (sizes_sorted * RADIUS_SCALE)
        spatial_bins = create_spatial_bins(u_ground, v_ground, r, d_ground, ground_points_fov, cell_size)


        ##### Depth Matching #####
        fitted_line_precalc_values = fit_line_on_plane_precalc(plane_normal, plane_offset) # optimization, calc outside of for loop
        
        step = min(r) * once_config["depth_map"]["lane_density_step_fraction"]
        min_spacing = max(r) * once_config["depth_map"]["min_spacing_fraction"]
        ###################################



        ########## Main Processing ##########
        output_positions = []
        for lane_pixels in lanes:
            lane_x = lane_pixels[:, 0]
            lane_y = lane_pixels[:, 1]


            ############# Intrinsics & Unit Vec #############
            intrinsic_points = pure_intrinsic_projection(lane_x, img_height - lane_y, fx, fy, cx, cy, once_config, distortion, None, plane_normal, plane_offset)

            model_type, model_params = fit_line_on_plane(
                intrinsic_points, 
                fitted_line_precalc_values, 
                curvature_threshold=once_config["projection_mapping"]["snap_curvature_threshold"]
            )
            #################################################
           


            ############# Depth Matching #############
            valid_depths = []
            valid_points = []
            matched_points = []
            seen = set()
            x_prev, y_prev = None, None
            for i in range(len(lane_x) - 1):
                x0, y0 = lane_x[i], lane_y[i]
                x1, y1 = lane_x[i + 1], lane_y[i + 1]
                dx, dy = x1 - x0, y1 - y0
                dist = np.hypot(dx, dy)
                num_steps = max(1, int(dist / step))
                t_vals = np.linspace(0, 1, num_steps)
                xs = x0 + t_vals * dx
                ys = y0 + t_vals * dy

                for x, y in zip(xs, ys):
                    if x_prev is not None and np.hypot(x - x_prev, y - y_prev) < min_spacing:
                        continue
                    x_prev, y_prev = x, y

                    if not (0 <= x < img_width and 0 <= y < img_height):
                        continue

                    x2, y2, depth, pt = query_depth_from_bins(spatial_bins, x, y, get_all_depths=False, prioritize_closest=True)
                    if depth is None:
                        continue

                    diff_eps = once_config["depth_map"]["diff_eps"]
                    x2r, y2r = round(x2 / diff_eps), round(y2 / diff_eps)
                    if (x2r, y2r) in seen:
                        continue
                    seen.add((x2r, y2r))

                    denom = dx * dx + dy * dy
                    if denom == 0:
                        xi, yi = x0, y0
                    else:
                        t = ((x2 - x0) * dx + (y2 - y0) * dy) / denom
                        t = np.clip(t, 0.0, 1.0)
                        xi, yi = x0 + t * dx, y0 + t * dy

                    valid_points.append((xi, yi))
                    valid_depths.append(depth)
                    matched_points.append(pt)

            if len(valid_points) > 0:
                min_valid_y = valid_points[-1][1]
                max_valid_y = valid_points[0][1]

                # --- FRONT END: anchors before first valid match ---
                front_indices = np.where(lane_y < min_valid_y)[0]
                if len(front_indices) > 0:
                    for i in front_indices:
                        xq, yq = lane_x[i], lane_y[i]
                        xq2, yq2, depth, pt = query_nearest_depth_from_bins(
                            spatial_bins, 
                            xq, yq, 
                            bin_search_range=once_config["depth_map"]["unmatched_max_search_range"]
                        )
                        if depth is None:
                            continue

                        # Determine projection reference
                        if i == 0:
                            # Project onto first segment (0→1)
                            x0, y0 = lane_x[0], lane_y[0]
                            x1, y1 = lane_x[1], lane_y[1]
                        else:
                            # Project onto segment (i-1→i)
                            x0, y0 = lane_x[i - 1], lane_y[i - 1]
                            x1, y1 = lane_x[i], lane_y[i]

                        vx, vy = x1 - x0, y1 - y0
                        denom = vx * vx + vy * vy
                        if denom == 0:
                            xi, yi = x0, y0
                        else:
                            t = ((xq2 - x0) * vx + (yq2 - y0) * vy) / denom
                            t = np.clip(t, 0.0, 1.0)
                            xi, yi = x0 + t * vx, y0 + t * vy

                        valid_points = np.vstack([valid_points, [xi, yi]])
                        depth = verify_with_intrinsic_projection(xi, yi, fx, fy, cx, cy, once_config, plane_normal, plane_offset, depth, distortion)
                        valid_depths = np.append(valid_depths, depth)
                        matched_points.append(pt)

                # --- BACK END: anchors after last valid match ---
                back_indices = np.where(lane_y > max_valid_y)[0]
                if len(back_indices) > 0:
                    for i in back_indices:
                        xq, yq = lane_x[i], lane_y[i]
                        xq2, yq2, depth, pt = query_nearest_depth_from_bins(
                            spatial_bins, 
                            xq, yq, 
                            bin_search_range=once_config["depth_map"]["unmatched_max_search_range"]
                        )
                        if depth is None:
                            continue

                        # Determine projection reference
                        if i == len(lane_x) - 1:
                            # Project onto last segment (-2→-1)
                            x0, y0 = lane_x[-2], lane_y[-2]
                            x1, y1 = lane_x[-1], lane_y[-1]
                        else:
                            # Project onto segment (i→i+1)
                            x0, y0 = lane_x[i], lane_y[i]
                            x1, y1 = lane_x[i + 1], lane_y[i + 1]

                        vx, vy = x1 - x0, y1 - y0
                        denom = vx * vx + vy * vy
                        if denom == 0:
                            xi, yi = x0, y0
                        else:
                            t = ((xq2 - x0) * vx + (yq2 - y0) * vy) / denom
                            t = np.clip(t, 0.0, 1.0)
                            xi, yi = x0 + t * vx, y0 + t * vy

                        valid_points = np.vstack([valid_points, [xi, yi]])
                        depth = verify_with_intrinsic_projection(xi, yi, fx, fy, cx, cy, once_config, plane_normal, plane_offset, depth, distortion)
                        valid_depths = np.append(valid_depths, depth)
                        matched_points.append(pt)

                valid_points = np.array(valid_points)
                valid_depths = np.array(valid_depths)
                depth_match_points = np.hstack([valid_points, valid_depths.reshape(-1,1)])
            else:
                depth_match_points = []
            ##########################################
        


            ############# 2D to 3D Matching #############
            if len(depth_match_points) > 2:
                # Learn mapping
                interp_s_to_t, pixel2d_sorted, _, _, _ = learn_pixel_to_t_mapping_for_lane(
                    depth_match_points,
                    model_type,
                    once_config,
                    model_params["z_intercept"], 
                    img_height, fx, fy, cx, cy, distortion,
                    linear_dir=model_params.get("line_dir"),                            # None if quadratic
                    quadratic_firstorder_dir=model_params.get("quad_firstorder_dir"),   # None if linear
                    quadratic_secondorder_dir=model_params.get("quad_secondorder_dir")  # None if linear
                )

                # Predict 3D for new pixels
                X3_pred, _, _ = predict_3d_for_s2_points(
                    lane_pixels, interp_s_to_t, pixel2d_sorted, model_type, 
                    model_params["z_intercept"],
                    linear_dir=model_params.get("line_dir"),                            # None if quadratic
                    quadratic_firstorder_dir=model_params.get("quad_firstorder_dir"),   # None if linear
                    quadratic_secondorder_dir=model_params.get("quad_secondorder_dir")  # None if linear
                )
                X3_pred = correct_z_mismatched_points(
                    X3_pred, 
                    intrinsic_points,
                    once_config, 
                    model_type, 
                    model_params["z_intercept"],
                    linear_dir=model_params.get("line_dir"),                            # None if quadratic
                    quadratic_firstorder_dir=model_params.get("quad_firstorder_dir"),   # None if linear
                    quadratic_secondorder_dir=model_params.get("quad_secondorder_dir")  # None if linear
                )
            else:
                X3_pred = intrinsic_points
                logger.warning(f"Not enough point-lidar matches to perform mapping [{len(depth_match_points)}/3]. Defaulting to intrinsic projection.")

            output_positions.append(X3_pred)
            #############################################
            

            total_lane_count += 1
        #####################################



        ########## File Saving ##########
        save_pointcloud_sequence( 
            output_positions,
            once_config["output"]["base_path"],
            seq_id,
            once_config["output"]["thesis_solution_path"],
            file_id
        )
        #################################
    
    return total_lane_count


In [ ]:
def run_all_solutions(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files):
    '''
    This function implements all solutions simultaneously (if that solution's config run is on):
        - Pure intrinsic
        - Intrinsic + ground plane
        - Lidar matching
        - Intrinsic + Lidar matching
        - Thesis code
    This function is rewritten to be optimal in sharing parameters between solutions when 
    available. This solution is always more optimal to run, but it does not give accurate 
    logging timing metrics (because it cannot differentiate runtime calls from each solution).
    For information on each individual solution, refer to that solution's function.

    This function checks if the output of thesis_code exists to determine whether to re-make
    a given set of input files. IE this checks if a file with <file_id> exists in thesis 
    output, to determine if the files need to be generated.

    Run a full processing pipeline for a single solution on a set of LiDAR files.
    This function iterates over each LiDAR file, performs any preprocessing steps, 
    applies the main processing routine to generate 3D outputs, and saves the results. 

    Args:
        once_calibration_path (str): Path to the calibration file used for processing.
        lane_detections_folder_path (str): Path to the folder containing the input 2D detection files.
        seq_id (str or int): The run id / sequence id. 
        lidar_files (list of Path): List of LiDAR file paths to process.

    Outputs:
        The processed 3D results are saved to disk. Each lane is saved as a separate 
        .bin file, named as <file_id>_<lane_idx>.bin, under the output path specified 
        in the configuration file for this solution. The first lane in a file has index 0,
        the next lane index 1, and so on.

    Returns:
        total_lane_count (int): the total count of lanes (attempted to) process.
    '''
    (
        T_camera_lidar,
        _,
        fx, fy, cx, cy,
        img_width, img_height,
        distortion
    ) = load_calibration(once_calibration_path)
    
    total_files = len(lidar_files)
    total_lane_count = 0

    dpi = once_config["depth_map"]["dpi"]
    RADIUS_SCALE = dpi / (72 * np.sqrt(np.pi))      # Convert dpi from point to px, set to radius
    cell_size = once_config["depth_map"]["lidar_point_size_max"] * RADIUS_SCALE
    max_search_range = once_config["depth_map"]["unmatched_max_search_range"]
    logger.info(f"Max search range: {cell_size * 2 *max_search_range}")


    for idx, once_lidar_path in enumerate(tqdm(lidar_files, desc=f"Processing all solutions LiDAR files for {seq_id}"), start=1):
        ########## Run Check ##########
        file_id = str(Path(once_lidar_path).stem)

        if once_config["runtime"]["file_id"] is not None:
            if file_id != once_config["runtime"]["file_id"]:
                continue
        elif not once_config["runtime"]["override_existing_output"] and Path(os.path.join(once_config["output"]["base_path"], seq_id, once_config["output"]["thesis_solution_path"], file_id + "_0.bin")).exists():
            logger.info(f"File [{idx}/{total_files}]: File already processed: {file_id}")
            continue
        else:
            logger.info(f"File [{idx}/{total_files}]: Processing:             {file_id}")
        ###############################



        ########## Preprocessing ##########

        ##### Load input 2D lanes #####
        lanes = load_2d_lanes(os.path.join(lane_detections_folder_path, file_id + ".txt"), once_config, img_width, img_height)
        if lanes is None or len(lanes) == 0:
            logger.error(f"No lanes are found.")
            continue


        ##### Calculate Ground Plane #####
        lidar_points, _, _, _ = get_lidar_points(once_lidar_path)

        # Transform points from lidar to camera frame
        points_lidar_hom = np.hstack((lidar_points, np.ones((lidar_points.shape[0], 1))))
        points_lidar_camera = (T_camera_lidar @ points_lidar_hom.T).T[:,:3] # lidar -> camera

        # Get ground plane
        prefiltered_points = prefilter_ground_plane(
            points_lidar_camera, 
            -once_config["ground_plane_ransac"]["camera_height"],
            once_config["ground_plane_ransac"]["prefilter_band_half_width"],
            vertical_axis=1
        )
        plane_normal, plane_offset, ground_plane_mask = estimate_ground_plane(
            prefiltered_points,
            once_config,
            vertical_axis=0
        )
        ground_points = prefiltered_points[ground_plane_mask]

        # Filter points into FOV
        u_ground, v_ground, d_ground, sizes_sorted, ground_points_fov = filter_and_clip_points(
            ground_points,
            fx, fy, cx, cy, img_width, img_height,
            once_config,
            distortion=distortion
        )

        # Create depth based search
        r = (sizes_sorted * RADIUS_SCALE)
        spatial_bins = create_spatial_bins(u_ground, v_ground, r, d_ground, ground_points_fov, cell_size)


        ##### Intrinsics  & Unit Vec #####
        fitted_line_precalc_values = fit_line_on_plane_precalc(plane_normal, plane_offset)
        classic_pure_intrinsic = []
        classic_intrinsic_ground_plane = []

        ##### Pure Depth #####
        classic_pure_depth = []
        classic_combined_intrinsic_depth = []

        ##### Depth Matching #####
        step = min(r) * once_config["depth_map"]["lane_density_step_fraction"]
        min_spacing = max(r) * once_config["depth_map"]["min_spacing_fraction"]

        ##### My Solution #####
        output_positions = []
        ###################################


        
        ########## Main Processing ##########
        for lane_pixels in lanes:
            lane_x = lane_pixels[:, 0]
            lane_y = lane_pixels[:, 1]


            ############# Intrinsics & Unit Vec #############
            if once_config["output"]["calc_cl_intrinsic"]:
                classic_pure_intrinsic.append(pure_intrinsic_projection(lane_x, img_height-lane_y, fx, fy, cx, cy, once_config, distortion, once_config["ground_plane_ransac"]["camera_height"]))
            

            if once_config["output"]["calc_cl_intrinsic_ground"] or once_config["output"]["calc_thesis_solution"]:
                intrinsic_points = pure_intrinsic_projection(lane_x, img_height - lane_y, fx, fy, cx, cy, once_config, distortion, None, plane_normal, plane_offset)
                classic_intrinsic_ground_plane.append(intrinsic_points)

                model_type, model_params = fit_line_on_plane(
                    intrinsic_points, 
                    fitted_line_precalc_values, 
                    curvature_threshold=once_config["projection_mapping"]["snap_curvature_threshold"]
                )
            #################################################



            ############# Pure Depth #############
            if once_config["output"]["calc_cl_depth"] or once_config["output"]["calc_cl_intrinsic_depth"]:
                matched_points = []
                udv_point = []
                for i in range(len(lane_x)):
                    _, _, depth, pt = query_nearest_depth_from_bins(spatial_bins, lane_x[i], lane_y[i], bin_search_range=None) #search infinitely
                    matched_points.append(pt)
                    udv_point.append(np.array([lane_x[i], lane_y[i], depth]))
                classic_pure_depth.append(np.array(matched_points))
                udv_point = np.array(udv_point)
                classic_combined_intrinsic_depth.append(project_pixel_to_depth(udv_point[:,0], img_height - udv_point[:,1], udv_point[:,2], fx, fy, cx, cy, distortion))
            ######################################



            ############# Depth Matching #############
            if once_config["output"]["calc_thesis_solution"]:
                valid_depths = []
                valid_points = []
                matched_points = []
                seen = set()
                x_prev, y_prev = None, None
                for i in range(len(lane_x) - 1):
                    x0, y0 = lane_x[i], lane_y[i]
                    x1, y1 = lane_x[i + 1], lane_y[i + 1]
                    dx, dy = x1 - x0, y1 - y0
                    dist = np.hypot(dx, dy)
                    num_steps = max(1, int(dist / step))
                    t_vals = np.linspace(0, 1, num_steps)
                    xs = x0 + t_vals * dx
                    ys = y0 + t_vals * dy

                    for x, y in zip(xs, ys):
                        if x_prev is not None and np.hypot(x - x_prev, y - y_prev) < min_spacing:
                            continue
                        x_prev, y_prev = x, y

                        if not (0 <= x < img_width and 0 <= y < img_height):
                            continue

                        x2, y2, depth, pt = query_depth_from_bins(spatial_bins, x, y, get_all_depths=False, prioritize_closest=True)
                        if depth is None:
                            continue

                        diff_eps = once_config["depth_map"]["diff_eps"]
                        x2r, y2r = round(x2 / diff_eps), round(y2 / diff_eps)
                        if (x2r, y2r) in seen:
                            continue
                        seen.add((x2r, y2r))

                        denom = dx * dx + dy * dy
                        if denom == 0:
                            xi, yi = x0, y0
                        else:
                            t = ((x2 - x0) * dx + (y2 - y0) * dy) / denom
                            t = np.clip(t, 0.0, 1.0)
                            xi, yi = x0 + t * dx, y0 + t * dy

                        valid_points.append((xi, yi))
                        valid_depths.append(depth)
                        matched_points.append(pt)

                if len(valid_points) > 0:
                    min_valid_y = valid_points[-1][1]
                    max_valid_y = valid_points[0][1]

                    # --- FRONT END: anchors before first valid match ---
                    front_indices = np.where(lane_y < min_valid_y)[0]
                    if len(front_indices) > 0:
                        for i in front_indices:
                            xq, yq = lane_x[i], lane_y[i]
                            xq2, yq2, depth, pt = query_nearest_depth_from_bins(
                                spatial_bins, 
                                xq, yq, 
                                bin_search_range=once_config["depth_map"]["unmatched_max_search_range"]
                            )
                            if depth is None:
                                continue

                            # Determine projection reference
                            if i == 0:
                                # Project onto first segment (0→1)
                                x0, y0 = lane_x[0], lane_y[0]
                                x1, y1 = lane_x[1], lane_y[1]
                            else:
                                # Project onto segment (i-1→i)
                                x0, y0 = lane_x[i - 1], lane_y[i - 1]
                                x1, y1 = lane_x[i], lane_y[i]

                            vx, vy = x1 - x0, y1 - y0
                            denom = vx * vx + vy * vy
                            if denom == 0:
                                xi, yi = x0, y0
                            else:
                                t = ((xq2 - x0) * vx + (yq2 - y0) * vy) / denom
                                t = np.clip(t, 0.0, 1.0)
                                xi, yi = x0 + t * vx, y0 + t * vy
                            
                            valid_points = np.vstack([valid_points, [xi, yi]])
                            depth = verify_with_intrinsic_projection(xi, yi, fx, fy, cx, cy, once_config, plane_normal, plane_offset, depth, distortion)
                            valid_depths = np.append(valid_depths, depth)
                            matched_points.append(pt)


                    # --- BACK END: anchors after last valid match ---
                    back_indices = np.where(lane_y > max_valid_y)[0]
                    if len(back_indices) > 0:
                        for i in back_indices:
                            xq, yq = lane_x[i], lane_y[i]
                            xq2, yq2, depth, pt = query_nearest_depth_from_bins(
                                spatial_bins, 
                                xq, yq, 
                                bin_search_range=once_config["depth_map"]["unmatched_max_search_range"]
                            )
                            if depth is None:
                                continue

                            # Determine projection reference
                            if i == len(lane_x) - 1:
                                # Project onto last segment (-2→-1)
                                x0, y0 = lane_x[-2], lane_y[-2]
                                x1, y1 = lane_x[-1], lane_y[-1]
                            else:
                                # Project onto segment (i→i+1)
                                x0, y0 = lane_x[i], lane_y[i]
                                x1, y1 = lane_x[i + 1], lane_y[i + 1]

                            vx, vy = x1 - x0, y1 - y0
                            denom = vx * vx + vy * vy
                            if denom == 0:
                                xi, yi = x0, y0
                            else:
                                t = ((xq2 - x0) * vx + (yq2 - y0) * vy) / denom
                                t = np.clip(t, 0.0, 1.0)
                                xi, yi = x0 + t * vx, y0 + t * vy

                            valid_points = np.vstack([valid_points, [xi, yi]])
                            depth = verify_with_intrinsic_projection(xi, yi, fx, fy, cx, cy, once_config, plane_normal, plane_offset, depth, distortion)
                            valid_depths = np.append(valid_depths, depth)
                            matched_points.append(pt)

                    valid_points = np.array(valid_points)
                    valid_depths = np.array(valid_depths)
                    depth_match_points = np.hstack([valid_points, valid_depths.reshape(-1,1)])
                else:
                    depth_match_points = []
            ##########################################
        


            ############# 2D to 3D Matching #############
            if once_config["output"]["calc_thesis_solution"]:
                if len(depth_match_points) > 2:
                    # Learn mapping
                    interp_s_to_t, pixel2d_sorted, _, _, _ = learn_pixel_to_t_mapping_for_lane(
                        depth_match_points,
                        model_type,
                        once_config,
                        model_params["z_intercept"], 
                        img_height, fx, fy, cx, cy, distortion,
                        linear_dir=model_params.get("line_dir"),                            # None if quadratic
                        quadratic_firstorder_dir=model_params.get("quad_firstorder_dir"),   # None if linear
                        quadratic_secondorder_dir=model_params.get("quad_secondorder_dir")  # None if linear
                    )

                    # Predict 3D for new pixels
                    X3_pred, _, _ = predict_3d_for_s2_points(
                        lane_pixels, interp_s_to_t, pixel2d_sorted, model_type, 
                        model_params["z_intercept"],
                        linear_dir=model_params.get("line_dir"),                            # None if quadratic
                        quadratic_firstorder_dir=model_params.get("quad_firstorder_dir"),   # None if linear
                        quadratic_secondorder_dir=model_params.get("quad_secondorder_dir")  # None if linear
                    )
                    X3_pred = correct_z_mismatched_points(
                        X3_pred, 
                        intrinsic_points,
                        once_config, 
                        model_type, 
                        model_params["z_intercept"],
                        linear_dir=model_params.get("line_dir"),                            # None if quadratic
                        quadratic_firstorder_dir=model_params.get("quad_firstorder_dir"),   # None if linear
                        quadratic_secondorder_dir=model_params.get("quad_secondorder_dir")  # None if linear
                    )
                else:
                    X3_pred = intrinsic_points
                    logger.warning(f"Not enough point-lidar matches to perform mapping [{len(depth_match_points)}/3]. Defaulting to intrinsic projection.")

                output_positions.append(X3_pred)
            #############################################


            total_lane_count += 1
        #####################################



        ############# File Saving #############
        if True:
            if once_config["output"]["calc_cl_intrinsic"]:
                save_pointcloud_sequence( 
                    classic_pure_intrinsic,
                    once_config["output"]["base_path"],
                    seq_id,
                    once_config["output"]["cl_intrinsic_path"],
                    file_id
                )
            if once_config["output"]["calc_cl_intrinsic_ground"]:
                save_pointcloud_sequence( 
                    classic_intrinsic_ground_plane,
                    once_config["output"]["base_path"],
                    seq_id,
                    once_config["output"]["cl_intrinsic_ground_path"],
                    file_id
                )
            if once_config["output"]["calc_cl_depth"]:
                save_pointcloud_sequence( 
                    classic_pure_depth,
                    once_config["output"]["base_path"],
                    seq_id,
                    once_config["output"]["cl_depth_path"],
                    file_id
                )
            if once_config["output"]["calc_cl_intrinsic_depth"]:
                save_pointcloud_sequence( 
                    classic_combined_intrinsic_depth,
                    once_config["output"]["base_path"],
                    seq_id,
                    once_config["output"]["cl_intrinsic_depth_path"],
                    file_id
                )
            if once_config["output"]["calc_thesis_solution"]:
                save_pointcloud_sequence( 
                    output_positions,
                    once_config["output"]["base_path"],
                    seq_id,
                    once_config["output"]["thesis_solution_path"],
                    file_id
                )
        #######################################
    
    return total_lane_count


In [ ]:
def main_per_seq(config, seq_id):
    '''
    This function implements all solutions for one run id / sequence id (if that solution's config run is on):
        - Pure intrinsic
        - Intrinsic + ground plane
        - Lidar matching
        - Intrinsic + Lidar matching
        - Thesis code

    If "run all no timing" is enabled, all solutions are run simultaneously (which is significantly faster). 
    Otherwise, runs one at a time (but accurately records runtime metrics for each). Thus, this function calls
    the sub-running functions per-solution (overriding data by settings).

    This function also establishes per-sequence logging. The final return value is per-solution runtimes.

    Args:
        config (dict): Configuration file.
        seq_id (str or int): The run id / sequence id. 

    Outputs:
        The processed 3D results are saved to disk per solution. See each solution's call function for specifics.
        Additionally outputs a log file (for the run/sequence) in the appropriate output file. 

    Returns:
        runtimes (dict): a dictionary of runtimes per solution.
    '''
    base_path = config["data"]["base_path"]

    lidar_folder_path = os.path.join(base_path, seq_id, config["data"]["lidar_path"])
    once_calibration_path = os.path.join(base_path, seq_id, config["data"]["calibration_path"])
    lane_detections_folder_path = os.path.join(base_path, seq_id, config["data"]["lane_detections_folder_path"])
    
    lidar_files = list(Path(lidar_folder_path).glob("*.bin"))

    switch_log_file(logger, os.path.join(config["output"]["base_path"], seq_id), seq_id)

    runtimes = {}
    if config["runtime"]["run_all_no_timing"]:
        logger.info("\n"*3 + ("="*100 + "\n")*3)
        logger.info("Starting all calculations.")
        start_timer(logger)
        lane_count = run_all_solutions(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files)
        logger.info("\n"*3)
        runtimes["run_all_no_timing"] = stop_timer(logger, "All simultaneously", lane_count)
    else:
        if config["output"]["calc_cl_intrinsic"]:
            logger.info("\n"*3 + ("="*100 + "\n")*3)
            logger.info("Starting intrinsic calculations.")
            start_timer(logger)
            lane_count = run_cl_intrinsic(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files)
            logger.info("\n"*3)
            runtimes[config["output"]["cl_intrinsic_path"]] = stop_timer(logger, "Pure Intrinsic", lane_count)

        if config["output"]["calc_cl_intrinsic_ground"]:
            logger.info("\n" + ("="*100 + "\n")*3)
            logger.info("Starting intrinsic ground plane calculations.")
            start_timer(logger)
            lane_count = run_cl_intrinsic_groundplane(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files)
            logger.info("\n"*3)
            runtimes[config["output"]["cl_intrinsic_ground_path"]] = stop_timer(logger, "Intrinsic + Ground Plane", lane_count)

        if config["output"]["calc_cl_depth"]:
            logger.info("\n" + ("="*100 + "\n")*3)
            logger.info("Starting depth calculations.")
            start_timer(logger)
            lane_count = run_cl_depth(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files)
            logger.info("\n"*3)
            runtimes[config["output"]["cl_depth_path"]] = stop_timer(logger, "Pure Depth", lane_count)

        if config["output"]["calc_cl_intrinsic_depth"]:
            logger.info("\n" + ("="*100 + "\n")*3)
            logger.info("Starting intrinsic + depth calculations.")
            start_timer(logger)
            lane_count = run_cl_intrinsic_depth(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files)
            logger.info("\n"*3)
            runtimes[config["output"]["cl_intrinsic_depth_path"]] = stop_timer(logger, "Intrinsic + Depth", lane_count)
            
        if config["output"]["calc_thesis_solution"]:
            logger.info("\n" + ("="*100 + "\n")*3)
            logger.info("Starting thesis code calculations.")
            start_timer(logger)
            lane_count = run_thesis_solution(once_calibration_path, lane_detections_folder_path, seq_id, lidar_files)
            logger.info("\n"*3)
            runtimes[config["output"]["thesis_solution_path"]] = stop_timer(logger, "Custom Thesis", lane_count)

    return runtimes


In [ ]:
def save_runtime_results(config, all_runtimes, seq_ids):
    '''
    Saves a dictionary of runtime results per sequence per solution. Overrides existing data if enabled,
    otherwise fills in missing runs / solutions.

    Args:
        config (dict): Configuration file.
        all_runtimes (dict): A dictionary of sequences with subdictionaries for solutions and their runtimes. 
        seq_id (strs or ints): All run ids / sequence ids.

    Outputs:
        The runtime results of all sequences and all solutions is saved to the disk. It appends to the JSON file,
        allowing async solution runs.
    '''
    runtime_output = os.path.join(config["output"]["base_path"], config["output"]["runtime_data_path"])
    if os.path.exists(runtime_output):
        with open(runtime_output, "r") as f:
            existing = json.load(f)
    else:
        existing = []

    existing_dict = {item["seq_id"]: item["runtimes"] for item in existing}
    for i, seq_id in enumerate(seq_ids):
        current_runtimes = {
            k: (v[i] if v[i] != -1 else None)
            for k, v in all_runtimes.items()
        }
        if seq_id not in existing_dict:
            # New seq, add
            existing_dict[seq_id] = current_runtimes
            continue
        # Existing seq, fill in values
        old_runtimes = existing_dict[seq_id]
        for key, old_val in old_runtimes.items():
            new_val = current_runtimes[key]
            if old_val is None and new_val is not None:
                # Fill previously-null value
                old_runtimes[key] = new_val
            elif config["runtime"]["override_existing_output"] and new_val is not None:
                # Override any existing value if allowed
                old_runtimes[key] = new_val
    output_list = [{"seq_id": seq_id, "runtimes": existing_dict[seq_id]} for seq_id in sorted(existing_dict.keys())]
    with open(runtime_output, "w") as f:
        json.dump(output_list, f, indent=2)

    main_logger.info(f"Runtime output file updated: {runtime_output}")


In [ ]:
######### Main Running Block #########

# Determine which seq ids to run on
base_path = once_config["data"]["base_path"]
seq_id = once_config["runtime"]["seq_id"]
seq_ids = []
if seq_id is None:
    # Get all sequences
    for name in os.listdir(base_path):
        full_path = os.path.join(base_path, name)
        if os.path.isdir(full_path):
            seq_ids.append(name)
    
    main_logger.info(f"Running all run ids: {len(seq_ids)} found.\n\n\n\n")
else:
    full_path = os.path.join(base_path, seq_id)
    if os.path.exists(full_path) and os.path.isdir(full_path):
        seq_ids.append(seq_id)
        main_logger.info(f"Running prespecified run id: {seq_id}")
    else:
        main_logger.error(f"Prespecified run id {seq_id} is not found, or not a run folder.")
        sys.exit(1)


# Run all sequences
runtime_keys = [
    "run_all_no_timing", 
    once_config["output"]["cl_intrinsic_path"],
    once_config["output"]["cl_intrinsic_ground_path"],
    once_config["output"]["cl_depth_path"],
    once_config["output"]["cl_intrinsic_depth_path"],
    once_config["output"]["thesis_solution_path"]
]
all_runtimes = {k: [] for k in runtime_keys}
for seq_id in seq_ids:
    main_logger.info(f"Running seq_id: {seq_id}")
    start_timer(main_logger)
    runtimes = main_per_seq(once_config, seq_id)
    stop_timer(main_logger, f"Seq {seq_id}", calc_lane_time=False)
    for k in runtime_keys:
        if k == "run_all_no_timing":
            value = runtimes.get(k, -1)
        else:
            value = runtimes.get(k, -1)
        all_runtimes[k].append(value)

# Save runtime results
save_runtime_results(once_config, all_runtimes, seq_ids)


Processing cl intrinsic LiDAR files for 000027: 100%|██████████| 1147/1147 [00:08<00:00, 142.77it/s]
Processing cl intrinsic ground plane LiDAR files for 000027: 100%|██████████| 1147/1147 [00:13<00:00, 82.20it/s] 
Processing cl depth LiDAR files for 000027: 100%|██████████| 1147/1147 [14:31<00:00,  1.32it/s] 
Processing cl intrinsic + depth LiDAR files for 000027: 100%|██████████| 1147/1147 [15:28<00:00,  1.23it/s] 
Processing thesis solution LiDAR files for 000027: 100%|██████████| 1147/1147 [00:40<00:00, 28.23it/s]
Processing cl intrinsic LiDAR files for 000028: 100%|██████████| 485/485 [00:03<00:00, 148.98it/s]
Processing cl intrinsic ground plane LiDAR files for 000028: 100%|██████████| 485/485 [00:05<00:00, 87.43it/s] 
Processing cl depth LiDAR files for 000028: 100%|██████████| 485/485 [00:51<00:00,  9.36it/s]
Processing cl intrinsic + depth LiDAR files for 000028: 100%|██████████| 485/485 [00:56<00:00,  8.66it/s]
Processing thesis solution LiDAR files for 000028: 100%|█████████